# Players dataframes with t-1 variables

En este script nos dedicaremos a crear una base de datos limpia segmentada por hitters y pitchers. Debido a que es totalmente análogo al script para variables del mismo periodo $t$, se omitirán solo las explicaciones del código reutilizado.

- **Visualización del contenido de las bases de datos.**
- **Limpieza de la base de datos y exportación.**
- **Creación de indicador de si el jugador es agente libre.**

Importemos los modulos necesarios así como especificar la configuración deseada.

In [79]:
import pandas as pd
import numpy as np
import math
import os
import warnings
import statsmodels.api as sm
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import MinMaxScaler
from termcolor import colored
print('Modulos importados')

Modulos importados


In [80]:
# Configuraciones
warnings.filterwarnings('ignore')

In [81]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('C:/Users/metal/Documents/Github/MLB_hardball_negotations/')

Directorio de trabajo previo: C:\Users\metal\Documents\Github\MLB_hardball_negotations


In [82]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = "C:/Users/metal/Documents/Github/MLB_hardball_negotations"
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

C:\Users\metal\Documents\Github\MLB_hardball_negotations
Nuevo directorio de trabajo: None


## Visualización de las bases de datos

### Equipos por estado

In [83]:
states = 'Data/Teams/team_states.csv'
df_states = pd.read_csv(states)

In [84]:
df_states.head()

,Estado,Cantidad de equipos
0,Alabama,0
1,Alaska,0
2,Arizona,1
3,Arkansas,0
4,California,5


### Acrónimos

Nos servirá como llave intermedia para unificar las bases de datos de los equipos

In [85]:
acronym = 'Data/Teams/team_acronym.csv'
df_acronym = pd.read_csv(acronym)

In [86]:
df_acronym.head()

,Equipo,Acronimo,Estado
0,Arizona Diamondbacks,ARI,Arizona
1,Atlanta Braves,ATL,Georgia
2,Baltimore Orioles,BAL,Maryland
3,Boston Red Sox,BOS,Massachusetts
4,Chicago Cubs,CHC,Illinois


Unamos esta dataframe con el de los equipos por estado

In [87]:
acronym_state = pd.merge(df_states, df_acronym, on = 'Estado')

In [163]:
acronym_state.head(30)

,Estado,Cantidad de equipos,Equipo,Acronimo
0,Arizona,1,Arizona Diamondbacks,ARI
1,California,5,Los Angeles Angels,LAA
2,California,5,Los Angeles Dodgers,LAD
3,California,5,Oakland Athletics,OAK
4,California,5,San Diego Padres,SD
5,California,5,San Francisco Giants,SF
6,Colorado,1,Colorado Rockies,COL
7,Florida,2,Miami Marlins,MIA
8,Florida,2,Tampa Bay Rays,TB
9,Georgia,1,Atlanta Braves,ATL


En este caso, el nombre de las variables es claro

In [89]:
# Definición de los dataframes para cada año
period = 12
teams_revenue = [None]*period

teams_revenue[12-1] = pd.DataFrame({
    'Team': ["COL", "BAL", "TEX", "MIN", "TB", "STL", "DET", "CIN", "CHW", "SD", 
             "MIA", "KC", "SEA", "BOS", "PIT", "NYY", "CLE", "NYM", "HOU", "OAK", 
             "LAD", "LAA", "WSH", "PHI", "ATL", "MIL", "TOR", "SF", "CHC", "ARI"],
    'Team_Payroll': [140012218, 44888388, 150037446, 149030158, 98342073, 156428325, 136287588, 115467321, 203205326, 224511694, 
                     82954422, 92613711, 115838907, 211812131, 56184032, 252957200, 66477492, 268292506, 183791796, 48443900, 
                     270381426, 179877811, 126809535, 244484097, 179938888, 130769325, 177013980, 162453046, 151054737, 85964090],
    'Year': [2022] * 30
})

teams_revenue[11-1] = pd.DataFrame({
    'Team': ["TB", "SF", "MIL", "OAK", "SEA", "CHW", "HOU", "TOR", "CIN", 
             "ATL", "DET", "LAD", "BOS", "MIA", "SD", "STL", "MIN", "COL", 
             "KC", "NYY", "PHI", "LAA", "CHC", "PIT", "BAL", "WSH", "TEX", 
             "ARI", "NYM"],
    'Team_Payroll': [70836327, 171890308, 99377415, 90400598, 83822113, 140926169, 194222042, 150140253, 50670534, 
                     152750691, 86348945, 265343390, 187100784, 58157900, 179764272, 151469994, 120084606, 116408966, 
                     91595545, 205669863, 197263223, 183849560, 144037170, 54356609, 42421870, 144415187, 95788819, 
                     91632929, 201189189],
    'Year': [2021] * 29
})

# Por ejemplo, para 2020:
teams_revenue[10-1] = pd.DataFrame({
    'Team': ["TB", "OAK", "ATL", "CHW", "SD", "CLE", "MIN", "LAD", "TOR", "MIA", 
             "BAL", "MIL", "CIN", "KC", "SF", "CHC", "PHI", "SEA", "LAA", "NYY", 
             "STL", "HOU", "DET", "WSH", "COL", "NYM", "ARI", "PIT", "BOS", "TEX"],
    'Team_Payroll': [28290689, 36720178, 63561931, 53665251, 73097954, 39299107, 55679689, 124917397, 54997060, 34222260, 
                     23478635, 41434086, 55535890, 34812194, 73408817, 86596171, 73543547, 51433829, 67040893, 111939081, 
                     73246343, 82890957, 43164880, 75067703, 67808533, 81945598, 65598752, 25337837, 84210390, 64214137],
    'Year': [2020] * 30
})

# Data for 2019
teams_revenue[9-1] = pd.DataFrame({
    'Team': ["TB", "MIN", "OAK", "HOU", "CLE", "ATL", "LAD", "ARI", "MIL", "WSH", 
             "NYY", "NYM", "PIT", "STL", "CHW", "TEX", "CIN", "PHI", "SD", "TOR", 
             "BAL", "COL", "SEA", "BOS", "LAA", "CHC", "MIA", "KC", "SF", "DET"],
    'Team_Payroll': [64178722, 125256003, 98644531, 167138259, 106838693, 143947963, 207000814, 119327905, 135889019, 172307808, 
                     223019037, 146835812, 72176474, 173762164, 91371201, 148538766, 128691569, 159637244, 103699790, 111371067, 
                     73316689, 157091013, 144391293, 229166880, 160270385, 221580085, 75596271, 104773003, 178582126, 114631137],
    'Year': [2019]*30
})

# Data for 2018
teams_revenue[8-1] = pd.DataFrame({
    'Team': ["OAK", "TB", "MIL", "HOU", "CLE", "ATL", "LAD", "ARI", "WSH", "NYY", 
             "NYM", "PIT", "BOS", "COL", "PHI", "STL", "CHW", "TEX", "CIN", "SD", 
             "TOR", "LAA", "SEA", "CHC", "MIA", "DET", "KC", "SF", "BAL"],
    'Team_Payroll': [80315288, 68810167, 108982016, 163524216, 143104703, 130849395, 199582045, 141724597, 181382609, 
                     180098151, 150187987, 91025861, 227398860, 143968544, 104297471, 163534311, 71839808, 140625018, 
                     100305768, 103773315, 150946147, 173717599, 160993827, 194259933, 91665500, 130959889, 129944821, 
                     205665348, 130413607],
    'Year': [2018]*29
})

# Data for 2017
teams_revenue[7-1] = pd.DataFrame({
    'Team': ["HOU", "CLE", "ARI", "MIL", "WSH", "MIN", "TB", "CHC", "COL", "OAK", 
             "NYY", "STL", "MIA", "LAD", "BOS", "CIN", "PIT", "ATL", "TEX", "SD", 
             "SEA", "CHW", "KC", "LAA", "PHI", "NYM", "TOR", "BAL", "DET", "SF"],
    'Team_Payroll': [138344211, 132258928, 102619475, 68945179, 182238854, 121048871, 77811205, 172092536, 139324341, 
                     73477080, 209044502, 145256085, 110765599, 259166393, 209872508, 96436003, 100503030, 105218575, 
                     163400840, 76921678, 160797330, 85495238, 148227822, 166161209, 104020808, 143990158, 175617487, 
                     175295707, 188498884, 189806809],
    'Year': [2017]*30
})

# Data for 2016
teams_revenue[6-1] = pd.DataFrame({
    'Team': ["STL", "CLE", "WSH", "MIL", "HOU", "CHC", "COL", "MIA", "PIT", "TB", 
             "TOR", "TEX", "SEA", "ARI", "BOS", "BAL", "CIN", "NYM", "SD", "OAK", 
             "CHW", "ATL", "MIN", "KC", "SF", "PHI", "DET", "LAA", "NYY", "LAD"],
    'Team_Payroll': [0, 105971268, 139383852, 62161191, 103328742, 184352494, 107902280, 77715306, 99518136, 63908549, 
                     148385479, 163021690, 146536553, 97874482, 207758836, 150248832, 88626437, 147738514, 83435559, 83109288, 
                     129715686, 96080633, 97244457, 141587358, 178834344, 103898809, 200179285, 174295919, 220743376, 266745494],
    'Year': [2016]*30
})

# Data for 2015
teams_revenue[5-1] = pd.DataFrame({
    'Team': ["PIT", "TOR", "HOU", "STL", "CHC", "KC", "ARI", "NYM", "TB", "TEX", 
             "MIN", "BAL", "LAA", "NYY", "OAK", "WSH", "SEA", "MIA", "SD", "COL", 
             "CHW", "BOS", "MIL", "DET", "CIN", "LAD", "ATL", "PHI"],
    'Team_Payroll': [99435606, 138309664, 81450835, 132178951, 132993810, 126529835, 76781801, 115698927, 76582652, 
                     156445607, 106431818, 117784476, 143348162, 221242659, 80376830, 165655095, 126045473, 76090525, 
                     107915272, 107102051, 118875487, 183481478, 94010873, 162160921, 116333097, 302735080, 110913266, 141722639],
    'Year': [2015]*28
})

# Data for 2014
teams_revenue[4-1] = pd.DataFrame({
    'Team': ["BAL", "PIT", "OAK", "LAA", "WSH", "CLE", "KC", "DET", "MIA", "STL", 
             "SEA", "MIL", "TOR", "NYM", "TB", "HOU", "LAD", "SF", "MIN", "CHW", 
             "COL", "CHC", "ATL", "SD", "CIN", "NYY", "ARI", "PHI", "BOS", "TEX"],
    'Team_Payroll': [113562943, 76974115, 92888412, 165244781, 143278577, 84481929, 98558300, 172143106, 50559679, 
                     121340470, 115534174, 108777046, 141812172, 94478037, 76821046, 51679490, 242157818, 160336782, 
                     87494478, 89297302, 96772365, 89539741, 116865707, 85001967, 113351318, 212512317, 106343190, 
                     175061605, 170095758, 139630583],
    'Year': [2014]*30
})

# Data for 2013
teams_revenue[3-1] = pd.DataFrame({
    'Team': ["OAK", "PIT", "TB", "ATL", "CLE", "BOS", "STL", "CIN", "DET", "BAL", 
             "KC", "TEX", "ARI", "WSH", "COL", "SD", "MIL", "LAA", "NYM", "SEA", 
             "TOR", "MIN", "MIA", "SF", "LAD", "CHC", "HOU", "CHW", "NYY", "PHI"],
    'Team_Payroll': [67055517, 71480984, 72600703, 94947950, 93008897, 170734264, 120192638, 106804640, 155032966, 
                     99630869, 85398434, 137937146, 88316478, 116584094, 79017288, 73286341, 88221045, 141542644, 
                     101300814, 82720022, 131771168, 96080633, 36209554, 140280186, 236694375, 100052899, 35077913, 
                     111777595, 235957526, 163536898],
    'Year': [2013]*30
})

# Data for 2012
teams_revenue[2-1] = pd.DataFrame({
    'Team': ["OAK", "WSH", "TB", "CIN", "BAL", "ATL", "TEX", "STL", "MIL", "ARI", 
             "CHW", "SF", "PIT", "SD", "LAA", "KC", "DET", "LAD", "NYY", "TOR", 
             "SEA", "CLE", "COL", "NYM", "MIA", "MIN", "PHI", "HOU", "BOS", "CHC"],
    'Team_Payroll': [60470192, 98044359, 69854135, 89901498, 82932168, 97928056, 138312903, 119957765, 110595795, 
                     82411013, 108841055, 137043205, 61118682, 63828925, 157168625, 70344235, 143782286, 117584717, 
                     226799742, 96031627, 84289928, 77057243, 87232529, 104988166, 98120191, 107939507, 178822491, 
                     54547239, 173216551, 116341526],
    'Year': [2012]*30
})

# Data for 2011
teams_revenue[1-1] = pd.DataFrame({
    'Team': ["Rays", "Rangers", "Diamondbacks", "Brewers", "Tigers", "Cardinals", "Blue Jays", "Braves", "Red Sox", 
             "Phillies", "Royals", "Reds", "Yankees", "Rockies", "Padres", "Pirates", "Athletics", "Dodgers", "Angels", 
             "Giants", "Orioles", "White Sox", "Mets", "Cubs", "Mariners", "Astros", "Twins"],
    'Team_Payroll': [45386925, 103741721, 65189970, 92277138, 111521015, 0, 73491604, 98696230, 170490761, 170018182, 
                     52072804, 81920546, 210950685, 82897934, 48018810, 48393645, 75655048, 115174517, 146648559, 
                     125242452, 94751399, 127263027, 142245841, 138083584, 101429148, 85323388, 113979594],
    'Year': [2011]*27
})

# Data for 2010
'''mlb_teams[0] = pd.DataFrame({
    'Team': ["TB", "STL", "CIN", "SD", "MIN", "TEX", "TOR", "COL", "SF", 
             "PHI", "KC", "BOS", "DET", "WSH", "LAA", "CLE", "LAD", "NYY", 
             "ARI", "HOU", "NYM", "BAL", "CHC", "PIT", "SEA"],
    'Team_Payroll': [73111908, 0, 76183915, 43679800, 88115519, 84947880, 72167275, 77433135, 98178333, 
                     137297541, 51104192, 157778541, 106838066, 48521480, 101667618, 51045206, 101450867, 
                     223310889, 61398736, 80184822, 127702202, 70097445, 131052537, 28219000, 84907671],
    'Year': [2010]*25
})'''

teams_revenue[0].head()

,Team,Team_Payroll,Year
0,Rays,45386925,2011
1,Rangers,103741721,2011
2,Diamondbacks,65189970,2011
3,Brewers,92277138,2011
4,Tigers,111521015,2011


In [90]:
for year in range(0,period):
    teams_revenue[year].rename(columns={'Team': 'Acronimo', 'Year': 'Anio', 'Team_Payroll': 'Ingresos_equipo'}, inplace=True)

In [91]:
# Crear el objeto scaler
scaler = MinMaxScaler()

for year in range(0,period):
    teams_revenue[year]['Ingresos_equipo'] = scaler.fit_transform(teams_revenue[year][['Ingresos_equipo']])

## Algoritmo para la creación de las bases de datos

A continuaicón, se optimizará el código para que se puedan obtener los *dataframes* anteriores para un cojuntos de datos de años secuenciales, como es nuestro caso

In [92]:
# Auxiliares:
free_agents = 'Data/Free_Agents/free_agents_'
hitting = 'Data/Statistics/Cumulative/Hitting/hitting_'
pitching = 'Data/Statistics/Cumulative/Pitching/pitching_'
salary = 'Data/Salary/salary_'
teams = 'ETL_Data/Transversal/Teams/free_agents_team_'
csv = '.csv'
period = 12
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period
df_panel_hitters = [None]*period
df_panel_pitchers = [None]*period

Leamos todos los archivos y creemos las copias

In [93]:
for year in range(0,period):    
    df_free_agents[year] = pd.read_csv(free_agents + str(2011 + year) + csv)
    df_hitting[year] = pd.read_csv(hitting + str(2011 + year) + csv)
    df_pitching[year] = pd.read_csv(pitching + str(2011 + year) + csv)
    df_salary[year] = pd.read_csv(salary + str(2011 + year) + csv)
    df_teams[year] = pd.read_csv(teams + str(2011 + year) + csv)
    
    df_free_agents_copy[year] = df_free_agents[year].copy()
    df_hitting_copy[year] = df_hitting[year].copy()
    df_pitching_copy[year] = df_pitching[year].copy()
    df_salary_copy[year] = df_salary[year].copy()
    df_teams_copy[year] = pd.read_csv(teams + str(2011 + year) + csv)

Tratemos las bases de datos por separado. Sin embargo, a todas les quitaremos la columna de rango y *Cash2023*.

Como no queremos que se repita la columna del año de la temporada de la base de datos, borremos la columna de *Year* de la base  de datos de los agentes libres. Como los años del contrato aparecen en la base de datos sobre los salarios, se prefiere conservar dicha columna en la base de datos de salarios puesto que esta base de datos es más general que la de los agentes libres, razón por la que se borrará de esta última base de datos. 

El equipo al que se cambia el agente libre está señalado por la columna del equipo en la base de datos de salarios y la estadísticas deportivas por lo que se borrará *Team From To* de la base de datos de los agentes libres. 

Como nos importan los salarios para este analisis, quitaremos la columna de los equipos en las bases de datos sobre las estadísticas deportivas de todos los jugadores, así como la posición que ocupan.

In [94]:
for year in range(0,period):
    # Drop columns:
    if any(name in df_free_agents_copy[year].columns for name in ['Rank','Pos','Year','Team From To']):
        df_free_agents_copy[year].drop('Rank', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Year', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Pos', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Team From To', axis = 1, inplace = True)
    if 'Rank' in df_salary_copy[year].columns:
        df_salary_copy[year].drop('Rank', axis = 1, inplace = True)
    if any(name in df_hitting_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_hitting_copy[year].drop('Rank', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Team', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Pos', axis = 1, inplace = True)
    if any(name in df_pitching_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_pitching_copy[year].drop('Rank', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Team', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Pos', axis = 1, inplace = True)

Debido a que aparecen columnas que inician con el  nombre *Unnamed*, tendremos que borrarlas con algún método general, el cual se muestra a continuación:

In [95]:
for year in range(0,period):
    # Base de datos de agentes libres:
    df_free_agents_copy[year].drop(df_free_agents_copy[year].columns[df_free_agents_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los salarios:
    df_salary_copy[year].drop(df_salary_copy[year].columns[df_salary_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los hitters:
    df_hitting_copy[year].drop(df_hitting_copy[year].columns[df_hitting_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los pitchers:
    df_pitching_copy[year].drop(df_pitching_copy[year].columns[df_pitching_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

Verifiquemos que ya no se encuentran dichas columnas molestas

In [96]:
df_free_agents_copy[9].columns

Index(['Player', 'Status', 'Team From', 'YRS', 'Value', 'AAV'], dtype='object')

In [97]:
df_salary_copy[11].columns

Index(['Player', 'Year', 'Pos', 'Team', 'BaseSalary', 'SigningBonus',
       'Payroll Salary', 'Adj Salary', 'Salary%', 'Cash', 'AAV', 'CONT YR',
       'CONT VALUE', 'Earnings', 'FA Year', 'Sign Age', 'Age', 'Weight',
       'Height'],
      dtype='object')

In [98]:
df_hitting_copy[2].columns

Index(['Player', 'GP', 'GP%', 'GS', 'GS%', 'AB', 'H', '2B', '3B', 'HR', 'RBI',
       'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'TVS'],
      dtype='object')

In [99]:
df_pitching_copy[5].columns

Index(['Player', 'GP', 'GS', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'W', 'L', 'SV',
       'WHIP', 'ERA', 'WAR', 'TVS'],
      dtype='object')

#### Agentes libres

No se conservará el equipo al que es contratado el agente libre puesto que esta información también la contiene la base de datos que facilita más el tratamiento _ETL_.

In [100]:
for year in range(0,period):
    df_free_agents_copy[year] = df_free_agents_copy[year].rename(columns = {'Player':'Jugador',
                                'Status':'Status_agente_libre', 'Team From':'Equipo_anterior',
                                'Value':'Valor_contrato', 'AAV':'Valor_promedio_contrato',
                                'YRS':'Anios_de_contrato'})
    
    free_agents_aux_1 = df_free_agents_copy[year]['Valor_contrato'].str.replace("$","")
    free_agents_aux_2 = free_agents_aux_1.str.replace(",","")
    free_agents_aux_3 = df_free_agents_copy[year]['Valor_promedio_contrato'].str.replace("$","")
    free_agents_aux_4 = free_agents_aux_3.str.replace(",","")
    df_free_agents_copy[year]['Valor_contrato'] = free_agents_aux_2
    df_free_agents_copy[year]['Valor_promedio_contrato'] = free_agents_aux_4
    
    df_free_agents_copy[year]['Valor_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_contrato'])
    df_free_agents_copy[year]['Valor_promedio_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_promedio_contrato'])

Observemos las dimensiones de las bases de datos como referencia

In [101]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 6)
(108, 6)
(213, 6)
(208, 6)
(221, 6)
(241, 6)
(100, 6)
(98, 6)
(105, 6)
(118, 6)
(141, 6)
(137, 6)


También el tipo de datos que contiene cada columna

In [102]:
df_free_agents_copy[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Jugador                  100 non-null    object
 1   Status_agente_libre      100 non-null    object
 2   Equipo_anterior          100 non-null    object
 3   Anios_de_contrato        100 non-null    int64 
 4   Valor_contrato           100 non-null    int64 
 5   Valor_promedio_contrato  100 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


#### Salarios

Como los salarios irán con las bases de datos de los _hitters_ y _pitchers_ es que se hará su proceso _ETL_ antes.

In [103]:
for year in range(0,period):
    # Cambio de nombres
    df_salary_copy[year] = df_salary_copy[year].rename(columns = {'Player':'Jugador',
                            'BaseSalary':'Sueldo_base', 'SigningBonus':'Bono_por_firma',
                            'Payroll Salary':'Sueldo_regular', 'Adj Salary':'Sueldo_ajustado',
                            'CONT YR':'Anios_de_contrato', 'CONT VALUE':'Valor_del_contrato',
                            'Earnings':'Ganancias', 'FA Year':'Anio_de_agente_libre',
                            'Sign Age':'Edad_al_firmar', 'Age':'Edad', 'Weight':'Peso',
                            'Height':'Altura', 'Year':'Anio', 'Pos':'Posicion',
                            'Salary%':'Sueldo_porcentual', 'Cash':'Pago_efectivo',
                            'AAV':'Valor_contrato_promedio', 'Team':'Acronimo'})
    
    # Tranformando al tipo de dato apropiado
    salary_aux_1 = df_salary_copy[year]['Sueldo_base'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[year]['Sueldo_base'] = salary_aux_2
    df_salary_copy[year]['Sueldo_base'] = pd.to_numeric(df_salary_copy[year]['Sueldo_base'])
    
    salary_aux_3 = df_salary_copy[year]['Sueldo_regular'].str.replace("$","")
    salary_aux_4 = salary_aux_3.str.replace(",","")
    df_salary_copy[year]['Sueldo_regular'] = salary_aux_4
    df_salary_copy[year]['Sueldo_regular'] = pd.to_numeric(df_salary_copy[year]['Sueldo_regular'])
    
    salary_aux_5 = df_salary_copy[year]['Sueldo_ajustado'].str.replace("$","")
    salary_aux_6 = salary_aux_5.str.replace(",","")
    df_salary_copy[year]['Sueldo_ajustado'] = salary_aux_6
    df_salary_copy[year]['Sueldo_ajustado'] = pd.to_numeric(df_salary_copy[year]['Sueldo_ajustado'])
    
    salary_aux_7 = df_salary_copy[year]['Valor_del_contrato'].str.replace("$","")
    salary_aux_8 = salary_aux_7.str.replace(",","")
    df_salary_copy[year]['Valor_del_contrato'] = salary_aux_8
    df_salary_copy[year]['Valor_del_contrato'] = pd.to_numeric(df_salary_copy[year]['Valor_del_contrato'])
    
    salary_aux_9 = df_salary_copy[year]['Bono_por_firma'].str.replace("$","")
    salary_aux_10 = salary_aux_9.str.replace(",","")
    df_salary_copy[year]['Bono_por_firma'] = salary_aux_10
    df_salary_copy[year]['Bono_por_firma'] = pd.to_numeric(df_salary_copy[year]['Bono_por_firma'])
    
    salary_aux_11 = df_salary_copy[year]['Ganancias'].str.replace("$","")
    salary_aux_12 = salary_aux_11.str.replace(",","")
    df_salary_copy[year]['Ganancias'] = salary_aux_12
    df_salary_copy[year]['Ganancias'] = pd.to_numeric(df_salary_copy[year]['Ganancias'])
    
    salary_aux_13 = df_salary_copy[year]['Pago_efectivo'].str.replace("$","")
    salary_aux_14 = salary_aux_13.str.replace(",","")
    df_salary_copy[year]['Pago_efectivo'] = salary_aux_14
    df_salary_copy[year]['Pago_efectivo'] = pd.to_numeric(df_salary_copy[year]['Pago_efectivo'])
    
    salary_aux_15 = df_salary_copy[year]['Valor_contrato_promedio'].str.replace("$","")
    salary_aux_16 = salary_aux_15.str.replace(",","")
    df_salary_copy[year]['Valor_contrato_promedio'] = salary_aux_16
    df_salary_copy[year]['Valor_contrato_promedio'] = pd.to_numeric(df_salary_copy[year]['Valor_contrato_promedio'])
    
    salary_aux_17 = df_salary_copy[year]['Altura'].str.replace("\"","")
    salary_aux_18 = salary_aux_17.str.replace("'","")
    df_salary_copy[year]['Altura'] = salary_aux_18
    df_salary_copy[year]['Altura'] = pd.to_numeric(df_salary_copy[year]['Altura'])/10
    # SUstitullamos los xeros
    height_mean = df_salary_copy[year]['Altura'].mean(skipna=True)
    df_salary_copy[year]['Altura'] = df_salary_copy[year].Altura.mask(df_salary_copy[year].Altura == 0, height_mean)
    
    df_salary_copy[year]['Anio_de_agente_libre'] = pd.to_numeric(df_salary_copy[year]['Anio_de_agente_libre'])
    df_salary_copy[year]['Anios_de_contrato'] = pd.to_numeric(df_salary_copy[year]['Anios_de_contrato'])
    df_salary_copy[year]['Edad'] = pd.to_numeric(df_salary_copy[year]['Edad'])

Por algunas particularidades de la base de datos, las columna que contiene la edad al firmar se tratará por separado aprovechando que la mayoría de los datos incorrectos tienen una longitud mayor a dos.

In [104]:
for year in range (0,period):
    df_salary_copy[year]['Edad_al_firmar'] = df_salary_copy[year]['Edad_al_firmar'].map(str)

    for edad in range(0,df_salary_copy[year].shape[0]):
        # String es mayor que 0:
        if len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) == 2:
            df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])
            
        # String es menor o igual que 0:
        elif len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != 2:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)    
    
        # Entero  menor a 0:
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)
         
    # Transformemos los datos a enteros
    df_salary_copy[year]['Edad_al_firmar'] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'])

Podemos verificar si se limpiaron adecuadamente las celdas de la columna de edades al firmar. Esto, al filtrar los datos que sean distintos a enteros y al observar si se pudo transformar toda la columna al tipo entero.

In [105]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != np.int64:
            print(type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]))

In [106]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            print(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])

In [107]:
#for year in range(0,period):
#    print(type(df_salary_copy[year][['Edad_al_firmar']].info()))

In [108]:
#for year in range(0,period):
#    print(year)
#    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
#        print(str(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) + ' ' + str(edad))

Por otro lado, falta corregir las entradas de las columnas de las edades que tengan valores menores a cero. Esto se hará de acuerdo al resto de columnas

In [109]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(df_salary_copy[year]['Edad'].iloc[edad])

0
-9
1
-9
2
-7


In [110]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year].shape[0]):
        # Condición para imputar:
        if df_salary_copy[year]['Edad'].iloc[edad] <= 0:
            # Si no se indica si tendrá año de agencia libre:
            if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                        ag_year = year + 2011 + 1
            # Si tendrá año de agencia libre
            else:
                ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
            # Get first year of contract
            ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
            # Años desde el el año inicial
            dif_years = year + 2011 - ini_year
            # Edad en la temporada:
            seasson_age = df_salary_copy[year]['Edad_al_firmar'].iloc[edad] + dif_years
            # Asignación
            df_salary_copy[year]['Edad'].iloc[edad] = seasson_age

Comprobemos que no hay ninguna edad negativa

In [111]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(str(df_salary_copy[year]['Edad'].iloc[edad]) + ' ' + str(edad))

Con la imputación de datos ya se puede crear la columna que contiene la antiguedad del agente libre bajo el contrato

In [112]:
for year in range(0,period):
    df_salary_copy[year]['Antiguedad'] = df_salary_copy[year]['Edad'] - df_salary_copy[year]['Edad_al_firmar']

Por último, convirtamos la columna del año a string para que se entienda como una categoría y no una variable numérica

In [113]:
for year in range(0,period):
    df_salary_copy[year]['Anio'] = df_salary_copy[year]['Anio'].map(str)

In [114]:
df_salary_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1045 non-null   object 
 1   Anio                     1045 non-null   object 
 2   Posicion                 1045 non-null   object 
 3   Acronimo                 1045 non-null   object 
 4   Sueldo_base              1045 non-null   int64  
 5   Bono_por_firma           1045 non-null   int64  
 6   Sueldo_regular           1045 non-null   int64  
 7   Sueldo_ajustado          1045 non-null   int64  
 8   Sueldo_porcentual        1045 non-null   float64
 9   Pago_efectivo            1045 non-null   int64  
 10  Valor_contrato_promedio  1045 non-null   int64  
 11  Anios_de_contrato        1045 non-null   int64  
 12  Valor_del_contrato       1045 non-null   int64  
 13  Ganancias                1045 non-null   int64  
 14  Anio_de_agente_libre    

#### Hitters

In [115]:
for year in range(0,period):
    # Cambio de nombres
    df_hitting_copy[year] = df_hitting_copy[year].rename(columns = {'Player':'Jugador',
                            'GP':'Juegos', 'GP%':'Porcentaje_juegos',
                            'AB':'At_bats', 'H':'Bateos', 'GS':'Juegos_iniciados',
                            'GS%':'Porcentaje_juegos_iniciados', 'RBI':'Runs_batted_in',
                            'HR':'Home_runs', 'AVG':'Bateos_promedio',
                            '2B':'Dobles', '3B':'Triples', 'OPS':'Porcentaje_On_base_plus_slugging',
                            'SLG':'Porcentaje_slugging', 'OBP':'Porcentaje_on_base'})

In [116]:
for year in range(0,period):
    print(df_hitting_copy[year].shape)

(521, 17)
(669, 17)
(734, 17)
(778, 17)
(809, 17)
(834, 17)
(810, 17)
(825, 17)
(850, 17)
(451, 17)
(753, 17)
(496, 17)


In [117]:
df_hitting_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Jugador                           834 non-null    object 
 1   Juegos                            834 non-null    int64  
 2   Porcentaje_juegos                 834 non-null    float64
 3   Juegos_iniciados                  834 non-null    int64  
 4   Porcentaje_juegos_iniciados       834 non-null    float64
 5   At_bats                           834 non-null    int64  
 6   Bateos                            834 non-null    int64  
 7   Dobles                            834 non-null    int64  
 8   Triples                           834 non-null    int64  
 9   Home_runs                         834 non-null    int64  
 10  Runs_batted_in                    834 non-null    int64  
 11  Bateos_promedio                   834 non-null    float64
 12  Porcenta

In [118]:
df_hitting_copy[5].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At_bats', 'Bateos', 'Dobles', 'Triples',
       'Home_runs', 'Runs_batted_in', 'Bateos_promedio', 'Porcentaje_on_base',
       'Porcentaje_slugging', 'Porcentaje_On_base_plus_slugging', 'WAR',
       'TVS'],
      dtype='object')

#### Pitchers

In [119]:
for year in range(0,period):
    # Cambio de nombres
    df_pitching_copy[year] = df_pitching_copy[year].rename(columns = {'Player':'Jugador',
                             'GP':'Juegos', 'GS':'Juegos_iniciados', 'IP':'Inning_pitched',
                             'H':'Bateos', 'R':'Carreras', 'ER':'Carreras_ganadas',
                             'BB':'Walks', 'SO':'Strike_outs', 'W':'Wins', 'L':'Losses',
                             'SV':'Saves'})

In [120]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 16)
(427, 16)
(485, 16)
(537, 16)
(586, 16)
(592, 16)
(581, 16)
(644, 16)
(668, 16)
(542, 16)
(611, 16)
(598, 16)


In [121]:
df_pitching_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           592 non-null    object 
 1   Juegos            592 non-null    int64  
 2   Juegos_iniciados  592 non-null    int64  
 3   Inning_pitched    592 non-null    float64
 4   Bateos            592 non-null    int64  
 5   Carreras          592 non-null    int64  
 6   Carreras_ganadas  592 non-null    int64  
 7   Walks             592 non-null    int64  
 8   Strike_outs       592 non-null    int64  
 9   Wins              592 non-null    int64  
 10  Losses            592 non-null    int64  
 11  Saves             592 non-null    int64  
 12  WHIP              592 non-null    float64
 13  ERA               592 non-null    float64
 14  WAR               587 non-null    float64
 15  TVS               592 non-null    float64
dtypes: float64(5), int64(10), object(1)
memory u

## Agregación de variables sugeridas por artículos

Las primeras variables que agregaremos son el cuadrado de todas las estadísticas deportivas, así como las siguientes variables:

- DOMINANCE = $/(Inning \; Pitched)$
- CONTROL = $Walks/(Inning \; Pitched)$
- COMMAND = $Strike-outs/Walks$

In [122]:
for year in range(0,period):
    df_pitching_copy[year]['Dominio'] = df_pitching_copy[year]['Strike_outs']/(df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Control'] = df_pitching_copy[year]['Walks']/(df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Comando'] = df_pitching_copy[year]['Strike_outs']/df_pitching_copy[year]['Walks']

In [123]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 19)
(427, 19)
(485, 19)
(537, 19)
(586, 19)
(592, 19)
(581, 19)
(644, 19)
(668, 19)
(542, 19)
(611, 19)
(598, 19)


In [124]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           485 non-null    object 
 1   Juegos            485 non-null    int64  
 2   Juegos_iniciados  485 non-null    int64  
 3   Inning_pitched    485 non-null    float64
 4   Bateos            485 non-null    int64  
 5   Carreras          485 non-null    int64  
 6   Carreras_ganadas  485 non-null    int64  
 7   Walks             485 non-null    int64  
 8   Strike_outs       485 non-null    int64  
 9   Wins              485 non-null    int64  
 10  Losses            485 non-null    int64  
 11  Saves             485 non-null    int64  
 12  WHIP              485 non-null    float64
 13  ERA               485 non-null    float64
 14  WAR               435 non-null    float64
 15  TVS               485 non-null    float64
 16  Dominio           484 non-null    float64
 1

Podemos verificar qué entradas poseen valores infinitos en la base de datos

In [125]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Siguiendo la sugerencia de algunos artículos, obtengamos el logaritmo de los salarios

In [126]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_base'] = np.log(df_salary_copy[year]['Sueldo_base'])
    df_salary_copy[year]['ln_Sueldo_ajustado'] = np.log(df_salary_copy[year]['Sueldo_ajustado'])
    df_salary_copy[year]['ln_Sueldo_regular'] = np.log(df_salary_copy[year]['Sueldo_regular'])

In [127]:
df_salary_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1013 non-null   object 
 1   Anio                     1013 non-null   object 
 2   Posicion                 1013 non-null   object 
 3   Acronimo                 1013 non-null   object 
 4   Sueldo_base              1013 non-null   int64  
 5   Bono_por_firma           1013 non-null   int64  
 6   Sueldo_regular           1013 non-null   int64  
 7   Sueldo_ajustado          1013 non-null   int64  
 8   Sueldo_porcentual        1013 non-null   float64
 9   Pago_efectivo            1013 non-null   int64  
 10  Valor_contrato_promedio  1013 non-null   int64  
 11  Anios_de_contrato        1013 non-null   int64  
 12  Valor_del_contrato       1013 non-null   int64  
 13  Ganancias                1013 non-null   int64  
 14  Anio_de_agente_libre    

Debido a que hay columnas con datos tipo _Nan_ o _NULL_, optaremos por imputarlos.

Mientras que los valores infinitos generados por las nuevas variables se sustituirán dependediendo del caso:

- 0/0: 0
- num/0: Máximo de la columna correspondientefijarán

In [128]:
for year in range(0,period):
    # Salaries
    mean_hgt = df_salary_copy[year].loc[df_salary_copy[year]['Altura'] > 4.9].Altura.mean()
    mean_wgh = df_salary_copy[year].loc[df_salary_copy[year]['Peso'] > 0].Peso.mean()
    df_salary_copy[year]['Altura'].fillna(value = mean_hgt, inplace = True)
    df_salary_copy[year]['Altura'].mask(df_salary_copy[year]['Altura'] <= 4.9, mean_hgt, inplace = True)
    df_salary_copy[year]['Peso'].fillna(value = mean_wgh, inplace = True)
    df_salary_copy[year]['Peso'].mask(df_salary_copy[year]['Peso'] <= 0, mean_wgh, inplace = True)
    
    # Pitchers
    mean_war = df_pitching_copy[year].loc[df_pitching_copy[year]['WAR'] > 0].WAR.mean()
    mean_dom = df_pitching_copy[year].loc[df_pitching_copy[year]['Dominio'] > 0].Dominio.mean()
    mean_con = df_pitching_copy[year].loc[df_pitching_copy[year]['Control'] > 0].Control.mean()
    mean_com = df_pitching_copy[year].loc[df_pitching_copy[year]['Comando'] > 0].Comando.mean()
    df_pitching_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_pitching_copy[year]['WAR'].mask(df_pitching_copy[year]['WAR'] <= 0, mean_war, inplace = True)
    df_pitching_copy[year]['Dominio'].fillna(value = mean_dom, inplace = True)
    df_pitching_copy[year]['Dominio'].mask(df_pitching_copy[year]['Dominio'] <= 0, mean_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = mean_con, inplace = True)
    df_pitching_copy[year]['Control'].mask(df_pitching_copy[year]['Control'] <= 0, mean_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = mean_com, inplace = True)
    df_pitching_copy[year]['Comando'].mask(df_pitching_copy[year]['Comando'] <= 0, mean_com, inplace = True)
    
    # Hitters
    mean_war = df_hitting_copy[year].loc[df_hitting_copy[year]['WAR'] > 0].WAR.mean()
    df_hitting_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_hitting_copy[year]['WAR'].mask(df_hitting_copy[year]['WAR'] <= 0, mean_war, inplace = True)

In [129]:
for year in range(0,period):   
    # Condiciones
    con_dom_1 = df_pitching_copy[year]['Strike_outs'] == 0
    con_con_1 = df_pitching_copy[year]['Walks'] == 0
    con_com_1 = df_pitching_copy[year]['Strike_outs'] == 0
                 
    # Imputación caso 0/0
    df_pitching_copy[year].loc[con_dom_1, "Dominio"] = 0
    df_pitching_copy[year].loc[con_con_1, "Control"] = 0
    df_pitching_copy[year].loc[con_com_1, "Comando"] = 0

In [130]:
for year in range(0,period):   
    # Máximos
    max_dom = df_pitching_copy[year]['Strike_outs'].max()/9
    max_con = df_pitching_copy[year]['Walks'].max()/9
    max_com = df_pitching_copy[year]['Strike_outs'].max()
    
    # Cambianfdo infinitos a NaNs
    df_pitching_copy[year]["Dominio"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Control"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Comando"].replace([np.inf, -np.inf], np.nan, inplace = True)
    
    # Imputación
    df_pitching_copy[year]['Dominio'].fillna(value = max_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = max_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = max_com, inplace = True)

Verifiquemos que ya no haya problemas con valores infinitos

In [131]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Así mismo, contemos los valores *NaN* que queden presentes

In [132]:
for year in range(0,period):
    print('Año: ' + str(2011 + year))
    print('Hitters:')
    df_hitting_copy[year].isna().sum()
    print('Pitchers:')
    df_pitching_copy[year].isna().sum()
    print('Free agents:')
    df_free_agents_copy[year].isna().sum()
    print('Salaries:')
    df_salary_copy[year].isna().sum()
    print("")

Año: 2011
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2012
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2013
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2014
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2015
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2016
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2017
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2018
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2019
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2020
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2021
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2022
Hitters:
Pitchers:
Free agents:
Salaries:



Ahora, repitamos este proceso para la base de datos de los salarios.

In [133]:
salary_names = ['ln_Sueldo_ajustado', 'ln_Sueldo_base', 'ln_Sueldo_regular']

In [134]:
for name in salary_names:
    print(name)
    
    for year in range(0,period):
        print(str(2011 + year))
        for element in range(0,len(df_salary_copy[year][name])):
            if df_salary_copy[year][name].iloc[element] <= 0:
                print(str(element) +  '  ' + str(df_salary_copy[year][name].iloc[element]))
        print("")

ln_Sueldo_ajustado
2011

2012

2013

2014

2015

2016

2017

2018
72  -inf
188  -inf

2019

2020

2021
6  -inf
55  -inf
166  -inf
274  -inf

2022
193  -inf

ln_Sueldo_base
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022

ln_Sueldo_regular
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



Al inspecsionar los errores nos damos cuenta que solo se desconocen los salarios ajustados y los fijaron a $0$. Usaremos el logaritmo base 10 del salario regular para sustituir dicho valor.

In [135]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_ajustado'].mask(df_salary_copy[year]['ln_Sueldo_ajustado'] < 0,
                                                    df_salary_copy[year]['ln_Sueldo_regular'],
                                                    inplace = True)

In [136]:
for year in range(0,period):
    print(str(2011 + year))
    for element in range(0,len(df_salary_copy[year]['ln_Sueldo_ajustado'])):
        if df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element] <= 0:
            print(str(element) +  '  ' + str(df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element]))
    print("")

2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



In [137]:
for year in range(0,period):
    print("Ajustado: " + str(df_salary_copy[year]['ln_Sueldo_ajustado'].mean())
          + 'n'
          + 'Regular: ' + str(df_salary_copy[year]['ln_Sueldo_regular'].mean()))

Ajustado: 13.785784770849762nRegular: 13.888874386883515
Ajustado: 13.81181498978195nRegular: 13.90039782504837
Ajustado: 13.54819591748778nRegular: 13.978975725734408
Ajustado: 13.66411021920866nRegular: 14.080738990191257
Ajustado: 13.49415866539551nRegular: 13.99344595993313
Ajustado: 13.478992231977141nRegular: 14.0261405412466
Ajustado: 13.575363447521962nRegular: 14.079742661574734
Ajustado: 13.562311948859815nRegular: 14.080756222584538
Ajustado: 13.521364643693584nRegular: 14.066559580453122
Ajustado: 13.029458522392627nRegular: 14.216780831796356
Ajustado: 13.70981799580226nRegular: 14.198804602895482
Ajustado: 13.924314685157658nRegular: 14.351734783233747


En efecto, ya no hay valores _NaN_ o _infinitos_.

Con el objetivo de hacer más eficiente la creación de las variables al cuadrado, lo haremos extrayendo el índice de las columnas de interés

In [138]:
df_hitting_copy[0].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At_bats', 'Bateos', 'Dobles', 'Triples',
       'Home_runs', 'Runs_batted_in', 'Bateos_promedio', 'Porcentaje_on_base',
       'Porcentaje_slugging', 'Porcentaje_On_base_plus_slugging', 'WAR',
       'TVS'],
      dtype='object')

In [139]:
df_pitching_copy[1].columns

Index(['Jugador', 'Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos',
       'Carreras', 'Carreras_ganadas', 'Walks', 'Strike_outs', 'Wins',
       'Losses', 'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
       'Comando'],
      dtype='object')

In [140]:
def get_col_indices(df, names):
    return df.columns.get_indexer(names)

In [141]:
hitting_names = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At_bats', 'Bateos',
                  'Dobles', 'Triples', 'Home_runs', 'Runs_batted_in', 'Bateos_promedio',
                  'Porcentaje_on_base', 'Porcentaje_slugging', 'TVS',
                  'Porcentaje_On_base_plus_slugging', 'WAR']	
pitching_names = ['Inning_pitched', 'Bateos', 'Carreras',
                  'Carreras_ganadas', 'Walks', 'Strike_outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando']

Con el objetivo de simplificar el código, verifiquemos si todos los índices en cada base de datos son los mismos

In [142]:
print('Hitters:')
for year in range(0,period):
    print(get_col_indices(df_hitting_copy[year], hitting_names))
    
print('Pitchers:')
for year in range(0,period):
    print(get_col_indices(df_pitching_copy[year], pitching_names))

Hitters:
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
Pitchers:
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3

In [143]:
hitting_indexes = list(get_col_indices(df_hitting_copy[0], hitting_names))
pitching_indexes = list(get_col_indices(df_pitching_copy[0], pitching_names))

In [144]:
for year in range(0,period):
    # Hitters:
    for hitter_name in hitting_indexes:
        df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name] + '_2'] = np.power(df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name]], 2)
    # Pitchers:
    for pitcher_name in pitching_indexes:
        df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name] + '_2'] = np.power(df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name]], 2)

Apreciemos el resultado final

In [145]:
df_hitting_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             734 non-null    object 
 1   Juegos                              734 non-null    int64  
 2   Porcentaje_juegos                   734 non-null    float64
 3   Juegos_iniciados                    734 non-null    int64  
 4   Porcentaje_juegos_iniciados         734 non-null    float64
 5   At_bats                             734 non-null    int64  
 6   Bateos                              734 non-null    int64  
 7   Dobles                              734 non-null    int64  
 8   Triples                             734 non-null    int64  
 9   Home_runs                           734 non-null    int64  
 10  Runs_batted_in                      734 non-null    int64  
 11  Bateos_promedio                     734 non-n

In [146]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Jugador             485 non-null    object 
 1   Juegos              485 non-null    int64  
 2   Juegos_iniciados    485 non-null    int64  
 3   Inning_pitched      485 non-null    float64
 4   Bateos              485 non-null    int64  
 5   Carreras            485 non-null    int64  
 6   Carreras_ganadas    485 non-null    int64  
 7   Walks               485 non-null    int64  
 8   Strike_outs         485 non-null    int64  
 9   Wins                485 non-null    int64  
 10  Losses              485 non-null    int64  
 11  Saves               485 non-null    int64  
 12  WHIP                485 non-null    float64
 13  ERA                 485 non-null    float64
 14  WAR                 485 non-null    float64
 15  TVS                 485 non-null    float64
 16  Dominio 

## Unión de las bases de datos
### Datos agregados por equipo

Solo resta añadir los datos relevantes al equipo al que pertenece cada jugador considerando la base de datos de la cantidad de equipos por estado

In [147]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
dtypes: float64(1), int64(7), object(2)
memory usage: 2.3+ KB


In [148]:
acronym_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Estado               30 non-null     object
 1   Cantidad de equipos  30 non-null     int64 
 2   Equipo               30 non-null     object
 3   Acronimo             30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [149]:
for year in range(0,period):
    df_teams_copy[year] = pd.merge(df_teams_copy[year], acronym_state, on = ['Equipo','Acronimo'])

In [150]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
 10  Estado                   28 non-null     object 
 11  Cantidad de equipos      28 non-null     int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 2.8+ KB


In [151]:
df_salary_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1036 non-null   object 
 1   Anio                     1036 non-null   object 
 2   Posicion                 1036 non-null   object 
 3   Acronimo                 1036 non-null   object 
 4   Sueldo_base              1036 non-null   int64  
 5   Bono_por_firma           1036 non-null   int64  
 6   Sueldo_regular           1036 non-null   int64  
 7   Sueldo_ajustado          1036 non-null   int64  
 8   Sueldo_porcentual        1036 non-null   float64
 9   Pago_efectivo            1036 non-null   int64  
 10  Valor_contrato_promedio  1036 non-null   int64  
 11  Anios_de_contrato        1036 non-null   int64  
 12  Valor_del_contrato       1036 non-null   int64  
 13  Ganancias                1036 non-null   int64  
 14  Anio_de_agente_libre    

Ahora, unamos las bases de datos sobre los equipos a las bases de datos de los salarios

In [152]:
for year in range(0,period):
    df_salary_copy[year] = pd.merge(df_teams_copy[year], df_salary_copy[year], on = 'Acronimo')

In [153]:
df_salary_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 0 to 1000
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   1001 non-null   object 
 1   Cantidad_agentes_libres  1001 non-null   int64  
 2   Valor_contrato_total     1001 non-null   int64  
 3   Acronimo                 1001 non-null   object 
 4   Victorias                1001 non-null   int64  
 5   Juegos totales           1001 non-null   int64  
 6   Playoffs                 1001 non-null   int64  
 7   Pennants won             1001 non-null   int64  
 8   WS ganadas               1001 non-null   int64  
 9   Promedio_victorias       1001 non-null   float64
 10  Estado                   1001 non-null   object 
 11  Cantidad de equipos      1001 non-null   int64  
 12  Jugador                  1001 non-null   object 
 13  Anio                     1001 non-null   object 
 14  Posicion                

In [154]:
df_salary_copy[1].head()

,Equipo,Cantidad_agentes_libres,Valor_contrato_total,Acronimo,Victorias,Juegos totales,Playoffs,Pennants won,WS ganadas,Promedio_victorias,...,Ganancias,Anio_de_agente_libre,Edad_al_firmar,Edad,Peso,Altura,Antiguedad,ln_Sueldo_base,ln_Sueldo_ajustado,ln_Sueldo_regular
0,Los Angeles Angels,4,321150000,LAA,89,162,10,1,1,0.549383,...,81521000,2015,29,32,215.16189,6.2415,3,16.951005,16.705882,17.080793
1,Los Angeles Angels,4,321150000,LAA,89,162,10,1,1,0.549383,...,102521000,2015,29,33,215.16189,6.2415,4,16.860033,17.001347,17.001347
2,Los Angeles Angels,4,321150000,LAA,89,162,10,1,1,0.549383,...,117550000,2013,32,35,205.00000,6.2000,3,16.733281,16.759950,16.759950
3,Los Angeles Angels,4,321150000,LAA,89,162,10,1,1,0.549383,...,135550000,2013,32,36,205.00000,6.2000,4,16.705882,16.733281,16.733281
4,Los Angeles Angels,4,321150000,LAA,89,162,10,1,1,0.549383,...,32342500,2017,29,29,210.00000,6.7000,0,16.454568,16.468753,16.468753


In [164]:
df_salary_copy[1]['Anio'].unique()

array(['2011', '2012'], dtype=object)

In [162]:
df_salary_copy[1]['Acronimo'].unique()

array(['LAA', 'DET', 'MIA', 'PHI', 'LAD', 'STL', 'MIL', 'COL', 'MIN',
       'NYM', 'TEX', 'WSH', 'ARI', 'TB', 'OAK', 'PIT', 'CHC', 'NYY',
       'BAL', 'BOS', 'CIN', 'KC', 'SF', 'ATL', 'CHW', 'HOU'], dtype=object)

In [155]:
teams_revenue[1].head()

,Acronimo,Ingresos_equipo,Anio
0,OAK,0.034385,2012
1,WSH,0.252520,2012
2,TB,0.088863,2012
3,CIN,0.205247,2012
4,BAL,0.164787,2012


In [158]:
teams_revenue[1]['Acronimo'].unique()

array(['OAK', 'WSH', 'TB', 'CIN', 'BAL', 'ATL', 'TEX', 'STL', 'MIL',
       'ARI', 'CHW', 'SF', 'PIT', 'SD', 'LAA', 'KC', 'DET', 'LAD', 'NYY',
       'TOR', 'SEA', 'CLE', 'COL', 'NYM', 'MIA', 'MIN', 'PHI', 'HOU',
       'BOS', 'CHC'], dtype=object)

In [165]:
for year in range(0,period):
    df_salary_copy[year] = pd.merge(teams_revenue[year], df_salary_copy[year], on =  'Acronimo')

In [167]:
df_salary_copy[1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1930 entries, 0 to 1929
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Acronimo                 1930 non-null   object 
 1   Ingresos_equipo          1930 non-null   float64
 2   Anio_x                   1930 non-null   int64  
 3   Equipo                   1930 non-null   object 
 4   Cantidad_agentes_libres  1930 non-null   int64  
 5   Valor_contrato_total     1930 non-null   int64  
 6   Victorias                1930 non-null   int64  
 7   Juegos totales           1930 non-null   int64  
 8   Playoffs                 1930 non-null   int64  
 9   Pennants won             1930 non-null   int64  
 10  WS ganadas               1930 non-null   int64  
 11  Promedio_victorias       1930 non-null   float64
 12  Estado                   1930 non-null   object 
 13  Cantidad de equipos      1930 non-null   int64  
 14  Jugador                 

Debido a que la mayoría de los jugadores juega tanto en la ofensiva como la defensiva es que tenemos que borrar los duplicados de la columna de la posición.

In [77]:
for year in range(0,period):
    df_hitting_copy[year] = pd.merge(df_hitting_copy[year], df_salary_copy[year], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitching_copy[year], df_salary_copy[year], on = 'Jugador')

In [78]:
for year in range(0,period):
    df_pitching_copy[year]['Porcentaje_juegos'] = df_pitching_copy[year]['Juegos']/df_pitching_copy[year]['Juegos totales']

In [79]:
for year in range(0,period):
    df_hitting_copy[year]['Ingresos_equipo_2'] = np.power(df_hitting_copy[year]['Ingresos_equipo'],2)
    df_pitching_copy[year]['Ingresos_equipo_2'] = np.power(df_pitching_copy[year]['Ingresos_equipo'],2)

In [80]:
df_hitting_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 66 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             0 non-null      object 
 1   Juegos                              0 non-null      int64  
 2   Porcentaje_juegos                   0 non-null      float64
 3   Juegos_iniciados                    0 non-null      int64  
 4   Porcentaje_juegos_iniciados         0 non-null      float64
 5   At_bats                             0 non-null      int64  
 6   Bateos                              0 non-null      int64  
 7   Dobles                              0 non-null      int64  
 8   Triples                             0 non-null      int64  
 9   Home_runs                           0 non-null      int64  
 10  Runs_batted_in                      0 non-null      int64  
 11  Bateos_promedio                     0 non-null      float64
 1

In [81]:
df_pitching_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 71 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  0 non-null      object 
 1   Juegos                   0 non-null      int64  
 2   Juegos_iniciados         0 non-null      int64  
 3   Inning_pitched           0 non-null      float64
 4   Bateos                   0 non-null      int64  
 5   Carreras                 0 non-null      int64  
 6   Carreras_ganadas         0 non-null      int64  
 7   Walks                    0 non-null      int64  
 8   Strike_outs              0 non-null      int64  
 9   Wins                     0 non-null      int64  
 10  Losses                   0 non-null      int64  
 11  Saves                    0 non-null      int64  
 12  WHIP                     0 non-null      float64
 13  ERA                      0 non-null      float64
 14  WAR                      0 non-null      fl

Para facilitar la observación de las trnasformaciones de manera más eficiente, ordenemos alfabéticamente la base de datos de acuerdo al nombre de las columnas.

In [82]:
for year in range(0,period):
    # Ordenando alfabéticamente
    df_salary_copy[year].sort_index(axis = 1, inplace = True)
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    df_free_agents_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando los índices
    df_salary_copy[year].reset_index(drop = True, inplace = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    df_free_agents_copy[year].reset_index(drop = True, inplace = True)

## Variables del periodo t-1

Lo que haremos será un *merge* de las bases de datos del año $t$ con el año $t-1$ sobre los jugadores. La razón de esto es que solo nos interesan los jugadores que han sido agentes libres por más de un año.

Si la primera base de datos es del año 2011, entonces tendremos que empezar en el año 2012. Creemos los dataframes que contendrán los datos para el modelo. Para que no se sobrepongan todos los periodos, crearemos dataframes auxiliares para guardar los nuevos datos

In [83]:
hitting_merge = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At_bats', 'Bateos',
                 'Dobles', 'Triples', 'Home_runs', 'Runs-batted-in', 'Bateos_promedio',
                 'Porcentaje_on-base', 'Porcentaje_slugging', 'TVS',
                 'Porcentaje_On_base_plus_slugging', 'WAR',
                 'Juegos_iniciados_2', 'Porcentaje_juegos_iniciados_2', 'At_bats_2', 'Bateos_2',
                 'Dobles_2', 'Triples_2', 'Home_runs_2', 'Runs_batted_in_2', 'Bateos_promedio_2',
                 'Porcentaje_on_base_2', 'Porcentaje_slugging_2', 'TVS_2',
                 'Porcentaje_On_base_plus_slugging_2', 'WAR_2']	
pitching_merge = ['Inning_pitched', 'Bateos_en_contra', 'Carreras_en_contra',
                  'Carreras_ganadas', 'Walks', 'Strike_outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando',
                  'Inning_pitched_2', 'Bateos_2', 'Carreras_2',
                  'Carreras_ganadas_2', 'Walks_2', 'Strike_outs_2', 'Wins_2', 'Losses_2',
                  'Saves_2', 'WHIP_2', 'ERA_2', 'WAR_2', 'TVS_2', 'Dominio_2', 'Control_2',
                  'Comando_2']

In [84]:
df_hitters_copy = [None]*period
df_pitchers_copy = [None]*period

In [85]:
for year in range(0,period):
    df_hitters_copy[year] = df_hitting_copy[year].copy()
    df_pitchers_copy[year] = df_pitching_copy[year].copy()

In [86]:
for year in range(1,period):    
    df_hitting_copy[year] = pd.merge(df_hitters_copy[year], df_hitters_copy[year-1], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitchers_copy[year], df_pitchers_copy[year-1], on = 'Jugador')

A continuación se verifica que la cantidad de columnas sea la misma, salvo por el primer periodo

In [87]:
"""for name in df_pitching_copy[11].columns:
    print(name)"""

'for name in df_pitching_copy[11].columns:\n    print(name)'

In [88]:
for year in range(0,period):
    print(df_hitting_copy[year].columns.shape)
    
for year in range(0,period):    
    print(df_pitching_copy[year].columns.shape)

(66,)
(131,)
(131,)
(131,)
(131,)
(131,)
(131,)
(131,)
(131,)
(131,)
(131,)
(131,)
(71,)
(141,)
(141,)
(141,)
(141,)
(141,)
(141,)
(141,)
(141,)
(141,)
(141,)
(141,)


In [89]:
for year in range(1,period):       
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('_x', '_t')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('_y', '_t_1')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('-', '_')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace(' ', '_')
    df_pitching_copy[year].drop(['ln_Sueldo_base_t_1', 'ln_Sueldo_ajustado_t_1', 'ln_Sueldo_regular_t_1'],
                           axis = 1, inplace = True)
    df_pitching_copy[year] = df_pitching_copy[year].sort_values(by = 'Jugador', ascending = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('_x', '_t')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('_y', '_t_1')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('-', '_')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace(' ', '_')
    df_hitting_copy[year].drop(['ln_Sueldo_base_t_1', 'ln_Sueldo_ajustado_t_1', 'ln_Sueldo_regular_t_1'],
                          axis = 1, inplace = True)
    df_hitting_copy[year] = df_hitting_copy[year].sort_values(by = 'Jugador', ascending = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    
    # Reordenando las columnas
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)

In [90]:
for name in df_pitching_copy[11].columns:
    print(name)

Acronimo_t
Acronimo_t_1
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_t
Anio_t_1
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
Bateos_2_t
Bateos_2_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Carreras_2_t
Carreras_2_t_1
Carreras_ganadas_2_t
Carreras_ganadas_2_t_1
Carreras_ganadas_t
Carreras_ganadas_t_1
Carreras_t
Carreras_t_1
Comando_2_t
Comando_2_t_1
Comando_t
Comando_t_1
Control_2_t
Control_2_t_1
Control_t
Control_t_1
Dominio_2_t
Dominio_2_t_1
Dominio_t
Dominio_t_1
ERA_2_t
ERA_2_t_1
ERA_t
ERA_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Edad_t_1
Equipo_t
Equipo_t_1
Estado_t
Estado_t_1
Ganancias_t
Ganancias_t_1
Ingresos_equipo_2_t
Ingresos_equipo_2_t_1
Ingresos_equipo_t
Ingresos_equipo_t_1
Inning_pitched_2_t
Inning_pitched_2_t_1
Inning_pitched_t
Inning_pitched_t_1
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_

Debido a que muchas de las variables del periodo $t_1$ pueden funcionar como controles más realistas, se optarán por dejarlas a excepción de la columna que contiene el dato del año al que hace referencia el dataframe del periodo $t_1$, es decir, la columna *Anio_t_1*. Esto se hará para *pitchers* y *hitters*. Por razones análogas, también de omitirá la columna que indica la cantidad de equipos en determinado estado ya que en el periodo de análisis es invariante.

Para facilitar la escritura del código, entenderemos la columna *Anio* como la columna *Anio_t*.

In [91]:
for year in range(1,period):
    df_pitching_copy[year].drop(['Anio_t_1', 'Estado_t_1', 'Edad_t_1'],
                           axis = 1, inplace = True)
    
    df_hitting_copy[year].drop(['Anio_t_1', 'Estado_t_1', 'Edad_t_1'],
                           axis = 1, inplace = True)
    
    # Reordenando las columnas
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando índice
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)

Cambiemos el súfijo de las basses de datos del año del 2011

In [92]:
year = 0
# Reiniciando los índices
df_hitting_copy[year] = df_hitting_copy[year].add_suffix('_t')
df_pitching_copy[year] = df_pitching_copy[year].add_suffix('_t')
# Corrección de columna del jugador
df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('Jugador_t', 'Jugador')
df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('Jugador_t', 'Jugador')

In [93]:
print("Salarios")
print(df_salary_copy[year].columns)
print("\n")
print("Hitters")
print(df_hitting_copy[year].columns)
print("\n")
print("Pitchers")
print(df_pitching_copy[year].columns)
print("\n")
print("Free agents")
print(df_free_agents_copy[year].columns)
print("\n")

Salarios
Index(['Acronimo', 'Altura', 'Anio', 'Anio_de_agente_libre',
       'Anios_de_contrato', 'Antiguedad', 'Bono_por_firma',
       'Cantidad de equipos', 'Cantidad_agentes_libres', 'Edad',
       'Edad_al_firmar', 'Equipo', 'Estado', 'Ganancias', 'Ingresos_equipo',
       'Juegos totales', 'Jugador', 'Pago_efectivo', 'Pennants won', 'Peso',
       'Playoffs', 'Posicion', 'Promedio_victorias', 'Sueldo_ajustado',
       'Sueldo_base', 'Sueldo_porcentual', 'Sueldo_regular',
       'Valor_contrato_promedio', 'Valor_contrato_total', 'Valor_del_contrato',
       'Victorias', 'WS ganadas', 'ln_Sueldo_ajustado', 'ln_Sueldo_base',
       'ln_Sueldo_regular'],
      dtype='object')


Hitters
Index(['Acronimo_t', 'Altura_t', 'Anio_t', 'Anio_de_agente_libre_t',
       'Anios_de_contrato_t', 'Antiguedad_t', 'At_bats_t', 'At_bats_2_t',
       'Bateos_t', 'Bateos_2_t', 'Bateos_promedio_t', 'Bateos_promedio_2_t',
       'Bono_por_firma_t', 'Cantidad de equipos_t',
       'Cantidad_agentes_libres

## Segmentación por Agentes libres

Separaremos los pitchers y hitters en dos grupos:

- Agentes libres.
- No agentes libres.

In [94]:
for year in range(0,period):
    # Filtrando los agentes libres
    df_hitters_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                            df_hitting_copy[year], on = 'Jugador')
    df_pitchers_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                             df_pitching_copy[year], on = 'Jugador')
    # FIltrando los que no son agentes libres
    df_hitters_no_free_agents[year] = df_hitting_copy[year][~df_hitting_copy[year].Jugador.isin(df_hitters_free_agents[year].Jugador)]
    df_pitchers_no_free_agents[year] = df_pitching_copy[year][~df_pitching_copy[year].Jugador.isin(df_pitchers_free_agents[year].Jugador)]
    
    # Reiniciando el índice
    df_hitters_free_agents[year] = df_hitters_free_agents[year].reindex(sorted(df_hitters_free_agents[year].columns), axis=1)
    df_pitchers_free_agents[year] = df_pitchers_free_agents[year].reindex(sorted(df_pitchers_free_agents[year].columns), axis=1)
    df_hitters_no_free_agents[year] = df_hitters_no_free_agents[year].reindex(sorted(df_hitters_no_free_agents[year].columns), axis=1)
    df_pitchers_no_free_agents[year] = df_pitchers_no_free_agents[year].reindex(sorted(df_pitchers_no_free_agents[year].columns), axis=1)    

Veamos los contenidos de las nuevas bases de datos

In [95]:
print("FA - Hitters:")
df_hitters_free_agents[9].info()
print("\n FA - Pitchers:")
df_pitchers_free_agents[9].info()
print("\n No FA - Hitters:")
df_hitters_no_free_agents[9].info()
print("\n No FA - Hitters:")
df_pitchers_no_free_agents[9].info()

FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 130 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(44), int64(75), object(11)
memory usage: 0.0+ bytes

 FA - Pitchers:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 140 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(48), int64(81), object(11)
memory usage: 0.0+ bytes

 No FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Columns: 125 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(44), int64(72), object(9)
memory usage: 0.0+ bytes

 No FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Columns: 135 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(48), int64(78), object(9)
memory usage: 0.0+ bytes


In [96]:
print("FA - Hitters:")
for year in range(0,period):
    print(df_hitters_free_agents[year].shape)
print("\n FA - Pitchers:")
for year in range(0,period):
    print(df_pitchers_free_agents[year].shape)

FA - Hitters:
(0, 71)
(0, 130)
(0, 130)
(0, 130)
(0, 130)
(0, 130)
(0, 130)
(0, 130)
(0, 130)
(0, 130)
(0, 130)
(0, 130)

 FA - Pitchers:
(0, 76)
(0, 140)
(0, 140)
(0, 140)
(0, 140)
(0, 140)
(0, 140)
(0, 140)
(0, 140)
(0, 140)
(0, 140)
(0, 140)


Por último, para facilitar futuras aplicaciones, pasemos todos los nombres de las columnas a miníscula

In [97]:
for year in range(0,period):
    df_hitters_free_agents[year].rename(columns = str.lower)
    df_pitchers_free_agents[year].rename(columns = str.lower)
    df_hitters_no_free_agents[year].rename(columns = str.lower)
    df_pitchers_no_free_agents[year].rename(columns = str.lower)

In [98]:
start_year = 2011
end_year = 2022
general_trans_path = 'ETL_Data/Transversal/Cumulative/'

In [99]:
for year in range(0,period):    
    # Exportemos los dataframes por separado
    df_hitters_free_agents[year].to_csv(general_trans_path + '/Hitters/Free_Agent/hitters_' + str(start_year + year) + '.csv',
                                        index = False)
    df_pitchers_free_agents[year].to_csv(general_trans_path + '/Fielders/Free_Agent/fielders_' + str(start_year + year) + '.csv',
                                         index = False)
    df_hitters_no_free_agents[year].to_csv(general_trans_path + '/Hitters/No_Free_Agent/hitters_' + str(start_year + year) + '.csv',
                                           index = False)
    df_pitchers_no_free_agents[year].to_csv(general_trans_path + '/Fielders/No_Free_Agent/fielders_' + str(start_year + year) + '.csv',
                                            index = False)

### Etiquetas para los agentes libres

Crearemos un etiqueta para indicar si el pitcher o hitter es  un agente libre o no.

## Panel Data

Con el objetivo de contar con una base de datos en estructura panel, uniremos las bases de datos

In [100]:
# Inicialización del panel
df_panel_all_hitter = df_hitting_copy[0]
df_panel_all_pitcher = df_pitching_copy[0]

for year in range(1,period):
    # Hitter
    df_panel_all_hitter = pd.concat([df_panel_all_hitter, df_hitting_copy[year]])
    
    # Pitcher
    df_panel_all_pitcher = pd.concat([df_panel_all_pitcher, df_pitching_copy[year]])

Veamos las estadísticas descriptivas de los panel

In [101]:
df_panel_all_hitter[['ln_Sueldo_ajustado_t']].describe()

,ln_Sueldo_ajustado_t
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [102]:
df_panel_all_pitcher.describe()

,Altura_t,Anio_de_agente_libre_t,Anios_de_contrato_t,Antiguedad_t,Bateos_t,Bateos_2_t,Bono_por_firma_t,Cantidad de equipos_t,Cantidad_agentes_libres_t,Carreras_t,...,WAR_2_t_1,WAR_t_1,WHIP_2_t_1,WHIP_t_1,WS_ganadas_t,WS_ganadas_t_1,Walks_2_t_1,Walks_t_1,Wins_2_t_1,Wins_t_1
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df_panel_all_hitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Columns: 129 entries, Acronimo_t to WS_ganadas_t_1
dtypes: float64(88), int64(32), object(9)
memory usage: 124.0+ bytes


In [104]:
df_panel_all_pitcher.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Columns: 139 entries, Acronimo_t to Wins_t_1
dtypes: float64(95), int64(35), object(9)
memory usage: 124.0+ bytes


Verifquemos que no haya problemas con valores *NaN* o *infinitos*

Valores *NaN*:

In [105]:
"""for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))"""

"for name in df_panel_all_hitter.columns:\n    print(name)\n    if type(name) != str:\n        for element in range(0,len(df_panel_all_hitter[name])):\n            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:\n                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))"

In [106]:
"""for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))"""

"for name in df_panel_all_pitcher.columns:\n    print(name)\n    if type(name) != str:\n        for element in range(0,len(df_panel_all_pitcher[name])):\n            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:\n                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))"

Valores *infinitos*

In [107]:
for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if math.isinf(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))

Acronimo_t
Altura_t
Anio_t
Anio_de_agente_libre_t
Anios_de_contrato_t
Antiguedad_t
At_bats_t
At_bats_2_t
Bateos_t
Bateos_2_t
Bateos_promedio_t
Bateos_promedio_2_t
Bono_por_firma_t
Cantidad de equipos_t
Cantidad_agentes_libres_t
Dobles_t
Dobles_2_t
Edad_t
Edad_al_firmar_t
Equipo_t
Estado_t
Ganancias_t
Home_runs_t
Home_runs_2_t
Ingresos_equipo_t
Ingresos_equipo_2_t
Juegos_t
Juegos totales_t
Juegos_iniciados_t
Juegos_iniciados_2_t
Jugador
Pago_efectivo_t
Pennants won_t
Peso_t
Playoffs_t
Porcentaje_On_base_plus_slugging_t
Porcentaje_On_base_plus_slugging_2_t
Porcentaje_juegos_t
Porcentaje_juegos_iniciados_t
Porcentaje_juegos_iniciados_2_t
Porcentaje_on_base_t
Porcentaje_on_base_2_t
Porcentaje_slugging_t
Porcentaje_slugging_2_t
Posicion_t
Promedio_victorias_t
Runs_batted_in_t
Runs_batted_in_2_t
Sueldo_ajustado_t
Sueldo_base_t
Sueldo_porcentual_t
Sueldo_regular_t
TVS_t
TVS_2_t
Triples_t
Triples_2_t
Valor_contrato_promedio_t
Valor_contrato_total_t
Valor_del_contrato_t
Victorias_t
WAR_t
WAR_2_

In [108]:
for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if math.isinf(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))

Acronimo_t
Altura_t
Anio_t
Anio_de_agente_libre_t
Anios_de_contrato_t
Antiguedad_t
Bateos_t
Bateos_2_t
Bono_por_firma_t
Cantidad de equipos_t
Cantidad_agentes_libres_t
Carreras_t
Carreras_2_t
Carreras_ganadas_t
Carreras_ganadas_2_t
Comando_t
Comando_2_t
Control_t
Control_2_t
Dominio_t
Dominio_2_t
ERA_t
ERA_2_t
Edad_t
Edad_al_firmar_t
Equipo_t
Estado_t
Ganancias_t
Ingresos_equipo_t
Ingresos_equipo_2_t
Inning_pitched_t
Inning_pitched_2_t
Juegos_t
Juegos totales_t
Juegos_iniciados_t
Jugador
Losses_t
Losses_2_t
Pago_efectivo_t
Pennants won_t
Peso_t
Playoffs_t
Porcentaje_juegos_t
Posicion_t
Promedio_victorias_t
Saves_t
Saves_2_t
Strike_outs_t
Strike_outs_2_t
Sueldo_ajustado_t
Sueldo_base_t
Sueldo_porcentual_t
Sueldo_regular_t
TVS_t
TVS_2_t
Valor_contrato_promedio_t
Valor_contrato_total_t
Valor_del_contrato_t
Victorias_t
WAR_t
WAR_2_t
WHIP_t
WHIP_2_t
WS ganadas_t
Walks_t
Walks_2_t
Wins_t
Wins_2_t
ln_Sueldo_ajustado_t
ln_Sueldo_base_t
ln_Sueldo_regular_t
Acronimo_t_1
Altura_t_1
Anio_de_agente

In [109]:
df_panel_all_hitter.sort_index(axis = 1, inplace = True)
df_panel_all_pitcher.sort_index(axis = 1, inplace = True)

Exportemos los paneles

In [110]:
general_panel_path = 'ETL_Data/Panel/Cumulative/Visualizations/'

In [111]:
df_panel_all_hitter.to_csv(general_panel_path + 'hitters_' + '.csv',
                           index = False)
df_panel_all_pitcher.to_csv(general_panel_path + 'fielders_' + '.csv',
                            index = False)

Repetiremos el procedimiento, pero únicamente para quienes son agentes libres

In [112]:
# Inicialización del panel
df_panel_fa_hitter = df_hitters_free_agents[0]
df_panel_fa_pitcher = df_pitchers_free_agents[0]

for year in range(1,period):
    # Hitter
    df_panel_fa_hitter = pd.concat([df_panel_fa_hitter, df_hitters_free_agents[year]])
    
    # Pitcher
    df_panel_fa_pitcher = pd.concat([df_panel_fa_pitcher, df_pitchers_free_agents[year]])

In [113]:
df_panel_fa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 134 entries, Acronimo_t to WS_ganadas_t_1
dtypes: float64(88), int64(35), object(11)
memory usage: 0.0+ bytes


In [114]:
df_panel_fa_pitcher.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 144 entries, Acronimo_t to Wins_t_1
dtypes: float64(95), int64(38), object(11)
memory usage: 0.0+ bytes


In [115]:
df_panel_fa_hitter.drop('Anios_de_contrato',
                        axis = 1, inplace = True)
df_panel_fa_pitcher.drop('Anios_de_contrato',
                         axis = 1, inplace = True)

In [116]:
df_panel_fa_hitter.sort_index(axis = 1, inplace = True)
df_panel_fa_pitcher.sort_index(axis = 1, inplace = True)

# Variables del Modelo Empírico

In [117]:
empiric_panel_hitter = df_panel_fa_hitter.copy()
empiric_panel_pitcher = df_panel_fa_pitcher.copy()

Veamos algunas estadísticas e información que contienen las bases de datos

In [118]:
print(empiric_panel_hitter.shape)

(0, 133)


In [119]:
print(empiric_panel_pitcher.shape)

(0, 143)


Las posiciones que hay en cada base de datos

In [120]:
empiric_panel_hitter['Posicion_t'].unique()

array([], dtype=object)

In [121]:
empiric_panel_pitcher['Posicion_t'].unique()

array([], dtype=object)

Ordenemos las bases de datos  por nombre y año

In [122]:
# Hitter
empiric_panel_hitter = empiric_panel_hitter.sort_values(by = ['Jugador','Anio_t'], ascending=True)
empiric_panel_hitter.reset_index(drop = True, inplace = True)

# Pitcher
empiric_panel_pitcher = empiric_panel_pitcher.sort_values(by = ['Jugador','Anio_t'], ascending=True)
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

In [123]:
empiric_panel_hitter[['Jugador','Anio_t']].head()

,Jugador,Anio_t


In [124]:
empiric_panel_pitcher[['Jugador','Anio_t']].head()

,Jugador,Anio_t


In [125]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

Obtengamos los índices de las columnas de interes

In [126]:
hitter_regular_stats = ['At_bats_2_t', 'At_bats_t',
                        'Bateos_2_t', 'Bateos_t',
                        'Bateos_promedio_2_t', 'Bateos_promedio_t',
                        'Dobles_2_t', 'Dobles_t',
                        'Home_runs_2_t', 'Home_runs_t',
                        'Ingresos_equipo_2_t', 'Ingresos_equipo_t',
                        'Juegos_iniciados_2_t', 'Juegos_iniciados_t', 
                        'Porcentaje_On_base_plus_slugging_2_t', 'Porcentaje_On_base_plus_slugging_t',
                        'Porcentaje_on_base_2_t', 'Porcentaje_on_base_t',
                        'Porcentaje_slugging_2_t', 'Porcentaje_slugging_t',
                        'Runs_batted_in_2_t', 'Runs_batted_in_t',
                        'Triples_2_t', 'Triples_t',
                        'WAR_2_t', 'WAR_t',
                        'At_bats_2_t_1', 'At_bats_t_1',
                        'Bateos_2_t_1', 'Bateos_t_1',
                        'Bateos_promedio_2_t_1', 'Bateos_promedio_t_1',
                        'Dobles_2_t_1', 'Dobles_t_1',
                        'Home_runs_2_t_1', 'Home_runs_t_1',
                        'Ingresos_equipo_2_t_1', 'Ingresos_equipo_t_1',
                        'Juegos_iniciados_2_t_1', 'Juegos_iniciados_t_1', 
                        'Porcentaje_On_base_plus_slugging_2_t_1', 'Porcentaje_On_base_plus_slugging_t_1',
                        'Porcentaje_on_base_2_t_1', 'Porcentaje_on_base_t_1',
                        'Porcentaje_slugging_2_t_1', 'Porcentaje_slugging_t_1',
                        'Runs_batted_in_2_t_1', 'Runs_batted_in_t_1',
                        'Triples_2_t_1', 'Triples_t_1',
                        'WAR_2_t_1', 'WAR_t_1']
hitter_regular_stats = sorted(hitter_regular_stats)

In [127]:
for name in empiric_panel_hitter.columns:
    print(name)

Acronimo_t
Acronimo_t_1
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_t
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
At_bats_2_t
At_bats_2_t_1
At_bats_t
At_bats_t_1
Bateos_2_t
Bateos_2_t_1
Bateos_promedio_2_t
Bateos_promedio_2_t_1
Bateos_promedio_t
Bateos_promedio_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad de equipos_t
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Dobles_2_t
Dobles_2_t_1
Dobles_t
Dobles_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Equipo_anterior
Equipo_t
Equipo_t_1
Estado_t
Ganancias_t
Ganancias_t_1
Home_runs_2_t
Home_runs_2_t_1
Home_runs_t
Home_runs_t_1
Ingresos_equipo_2_t
Ingresos_equipo_2_t_1
Ingresos_equipo_t
Ingresos_equipo_t_1
Juegos totales_t
Juegos_iniciados_2_t
Juegos_iniciados_2_t_1
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_t_1
Juegos_totales_t
Juegos_totales_t_1
Jugador
Pago_efectivo_t
Pago_efectivo_t_1
Pennants

In [128]:
# Hitter
for stat in range(0,len(hitter_regular_stats)):
    # Variables auxiliares
    stat_name = hitter_regular_stats[stat]
    max_stat_name = stat_name + '_H'
    min_stat_name = stat_name + '_L'
    
    # Máximos por equipo
    max_stat = pd.DataFrame({"Acronimo_t":empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].max().index,
                             max_stat_name: empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].max().values})
    # Mínimos por equipo
    min_stat = pd.DataFrame({"Acronimo_t":empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].min().index,
                             min_stat_name: empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].min().values})
    empiric_panel_hitter = empiric_panel_hitter.merge(max_stat, on = "Acronimo_t",
                                                      how = "left")
    empiric_panel_hitter = empiric_panel_hitter.merge(min_stat, on = "Acronimo_t",
                                                      how = "left")

In [129]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

In [130]:
start = empiric_panel_hitter.columns.get_loc('At_bats_2_t_1_H')
end = empiric_panel_hitter.columns.get_loc('WAR_t_1_L') + 1
empiric_panel_hitter.iloc[:,start:end]

,At_bats_2_t_1_H,At_bats_2_t_1_L,At_bats_t_H,At_bats_t_L,At_bats_t_1_H,At_bats_t_1_L,Bateos_2_t_H,Bateos_2_t_L,Bateos_2_t_1_H,Bateos_2_t_1_L,...,Triples_t_1_L,WAR_2_t_H,WAR_2_t_L,WAR_2_t_1_H,WAR_2_t_1_L,WAR_t_H,WAR_t_L,WAR_t_1_H,Acronimo_t,WAR_t_1_L


Repitamos el mismo proceso para los lanzadores

In [131]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

Repitamos el mismo proceso para los filderos

In [132]:
pitcher_regular_stats = ['Bateos_2', 'Bateos',
                        'Carreras_2', 'Carreras',
                        'Carreras_ganadas_2', 'Carreras_ganadas',
                        'Comando_2', 'Comando',
                        'Control_2', 'Control',
                        'Dominio_2', 'Dominio', 
                        'ERA_2', 'ERA',
                        'Ingresos_equipo_2', 'Ingresos_equipo',
                        'Inning_pitched_2', 'Inning_pitched',
                        'Losses_2', 'Losses',
                        'Saves_2', 'Saves',
                        'Strike_outs_2', 'Strike_outs',
                        'WAR_2', 'WAR',
                        'WHIP_2', 'WHIP',
                        'Walks_2', 'Walks',
                        'Wins_2', 'Wins']

# Agregar sufijo "_t" a cada variable
pitcher_regular_stats_t = [var + "_t" for var in pitcher_regular_stats]

# Agregar sufijo "_t_1" a cada variable
pitcher_regular_stats_t_1 = [var + "_t_1" for var in pitcher_regular_stats]

pitcher_regular_stats = pitcher_regular_stats_t + pitcher_regular_stats_t_1
pitcher_regular_stats = sorted(pitcher_regular_stats)

print(pitcher_regular_stats)

['Bateos_2_t', 'Bateos_2_t_1', 'Bateos_t', 'Bateos_t_1', 'Carreras_2_t', 'Carreras_2_t_1', 'Carreras_ganadas_2_t', 'Carreras_ganadas_2_t_1', 'Carreras_ganadas_t', 'Carreras_ganadas_t_1', 'Carreras_t', 'Carreras_t_1', 'Comando_2_t', 'Comando_2_t_1', 'Comando_t', 'Comando_t_1', 'Control_2_t', 'Control_2_t_1', 'Control_t', 'Control_t_1', 'Dominio_2_t', 'Dominio_2_t_1', 'Dominio_t', 'Dominio_t_1', 'ERA_2_t', 'ERA_2_t_1', 'ERA_t', 'ERA_t_1', 'Ingresos_equipo_2_t', 'Ingresos_equipo_2_t_1', 'Ingresos_equipo_t', 'Ingresos_equipo_t_1', 'Inning_pitched_2_t', 'Inning_pitched_2_t_1', 'Inning_pitched_t', 'Inning_pitched_t_1', 'Losses_2_t', 'Losses_2_t_1', 'Losses_t', 'Losses_t_1', 'Saves_2_t', 'Saves_2_t_1', 'Saves_t', 'Saves_t_1', 'Strike_outs_2_t', 'Strike_outs_2_t_1', 'Strike_outs_t', 'Strike_outs_t_1', 'WAR_2_t', 'WAR_2_t_1', 'WAR_t', 'WAR_t_1', 'WHIP_2_t', 'WHIP_2_t_1', 'WHIP_t', 'WHIP_t_1', 'Walks_2_t', 'Walks_2_t_1', 'Walks_t', 'Walks_t_1', 'Wins_2_t', 'Wins_2_t_1', 'Wins_t', 'Wins_t_1']


In [133]:
# Pitcheer
for stat in range(0,len(pitcher_regular_stats)):
    # Variables auxiliares
    stat_name = pitcher_regular_stats[stat]
    max_stat_name = stat_name + '_H'
    min_stat_name = stat_name + '_L'
    
    # Máximos por equipo
    max_stat = pd.DataFrame({"Acronimo_t":empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].max().index,
                             max_stat_name: empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].max().values})
    # Mínimos por equipo
    min_stat = pd.DataFrame({"Acronimo_t":empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].min().index,
                             min_stat_name: empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].min().values})
    empiric_panel_pitcher = empiric_panel_pitcher.merge(max_stat, on = "Acronimo_t",
                                                        how = "left")
    empiric_panel_pitcher = empiric_panel_pitcher.merge(min_stat, on = "Acronimo_t",
                                                        how = "left")

In [134]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

In [135]:
start = empiric_panel_pitcher.columns.get_loc('Bateos_2_t_1_H')
end = empiric_panel_pitcher.columns.get_loc('Wins_t_1_L') + 1
empiric_panel_pitcher.iloc[:,start:end]

,Bateos_2_t_1_H,Bateos_2_t_1_L,Bateos_t_H,Bateos_t_L,Bateos_t_1_H,Bateos_t_1_L,Carreras_2_t_H,Carreras_2_t_L,Carreras_2_t_1_H,Carreras_2_t_1_L,...,Walks_t_1_L,Wins_2_t_H,Wins_2_t_L,Wins_2_t_1_H,Wins_2_t_1_L,Wins_t_H,Wins_t_L,Wins_t_1_H,Acronimo_t,Wins_t_1_L


Lo que se hará ahora es agregar todos los años (12) para cada agente en la base de datos y rellenar los *NaN* faltantes con 0 ya que representan la ausencia de desempeño en este caso.

In [136]:
# Hitter
empiric_panel_hitter = empiric_panel_hitter["Anio_t"].drop_duplicates().to_frame().merge(empiric_panel_hitter["Jugador"].drop_duplicates(),
                                                                                         how = "cross").merge(empiric_panel_hitter,
                                                                                                              how = "left")
empiric_panel_hitter.reset_index(drop = True, inplace = True)

# Pitcher
empiric_panel_pitcher = empiric_panel_pitcher["Anio_t"].drop_duplicates().to_frame().merge(empiric_panel_pitcher["Jugador"].drop_duplicates(),
                                                                                           how = "cross").merge(empiric_panel_pitcher,
                                                                                                                how = "left")
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

Veamos las dimensiones

In [137]:
empiric_panel_hitter.shape

(0, 237)

In [138]:
empiric_panel_pitcher.shape

(0, 271)

Para ser consistentes, las columnas que contienen datos de tipo *string* las imputaremos con la palabra *No* ya que representará que no tenía equipo, ni posición, etc.

In [139]:
empiric_panel_hitter.select_dtypes(include =['object'],
                                   exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Acronimo_t_1         0 non-null      object
 1   Anio_t               0 non-null      object
 2   Equipo_anterior      0 non-null      object
 3   Equipo_t             0 non-null      object
 4   Equipo_t_1           0 non-null      object
 5   Estado_t             0 non-null      object
 6   Jugador              0 non-null      object
 7   Posicion_t           0 non-null      object
 8   Posicion_t_1         0 non-null      object
 9   Status_agente_libre  0 non-null      object
 10  Acronimo_t           0 non-null      object
dtypes: object(11)
memory usage: 124.0+ bytes


In [140]:
empiric_panel_hitter[['Acronimo_t',
                      'Equipo_anterior',
                      'Equipo_t',
                      'Estado_t',
                      'Posicion_t',
                      'Status_agente_libre',
                      'Acronimo_t_1',
                      'Equipo_t_1',
                      'Posicion_t_1']] = \
empiric_panel_hitter[['Acronimo_t',
                      'Equipo_anterior',
                      'Equipo_t',
                      'Estado_t',
                      'Posicion_t',
                      'Status_agente_libre',
                      'Acronimo_t_1',
                      'Equipo_t_1',
                      'Posicion_t_1']].fillna('No')

Veamos si funcionó la imputación

In [141]:
empiric_panel_hitter.select_dtypes(include =['object'],
                                   exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Acronimo_t_1         0 non-null      object
 1   Anio_t               0 non-null      object
 2   Equipo_anterior      0 non-null      object
 3   Equipo_t             0 non-null      object
 4   Equipo_t_1           0 non-null      object
 5   Estado_t             0 non-null      object
 6   Jugador              0 non-null      object
 7   Posicion_t           0 non-null      object
 8   Posicion_t_1         0 non-null      object
 9   Status_agente_libre  0 non-null      object
 10  Acronimo_t           0 non-null      object
dtypes: object(11)
memory usage: 124.0+ bytes


Ahora con los lanzadores

In [142]:
empiric_panel_pitcher.select_dtypes(include =['object'],
                                    exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Acronimo_t_1         0 non-null      object
 1   Anio_t               0 non-null      object
 2   Equipo_anterior      0 non-null      object
 3   Equipo_t             0 non-null      object
 4   Equipo_t_1           0 non-null      object
 5   Estado_t             0 non-null      object
 6   Jugador              0 non-null      object
 7   Posicion_t           0 non-null      object
 8   Posicion_t_1         0 non-null      object
 9   Status_agente_libre  0 non-null      object
 10  Acronimo_t           0 non-null      object
dtypes: object(11)
memory usage: 124.0+ bytes


In [143]:
empiric_panel_pitcher[['Acronimo_t',
                       'Equipo_anterior',
                       'Equipo_t',
                       'Estado_t',
                       'Posicion_t',
                       'Status_agente_libre',
                       'Acronimo_t_1',
                       'Equipo_t_1',
                       'Posicion_t_1']] = \
empiric_panel_pitcher[['Acronimo_t',
                       'Equipo_anterior',
                       'Equipo_t',
                       'Estado_t',
                       'Posicion_t',
                       'Status_agente_libre',
                       'Acronimo_t_1',
                       'Equipo_t_1',
                       'Posicion_t_1']].fillna('No')

In [144]:
empiric_panel_pitcher.select_dtypes(include =['object'],
                                    exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Acronimo_t_1         0 non-null      object
 1   Anio_t               0 non-null      object
 2   Equipo_anterior      0 non-null      object
 3   Equipo_t             0 non-null      object
 4   Equipo_t_1           0 non-null      object
 5   Estado_t             0 non-null      object
 6   Jugador              0 non-null      object
 7   Posicion_t           0 non-null      object
 8   Posicion_t_1         0 non-null      object
 9   Status_agente_libre  0 non-null      object
 10  Acronimo_t           0 non-null      object
dtypes: object(11)
memory usage: 124.0+ bytes


En ambos casos, la imputación fue un éxito. No importa que en periodos de contratación de haya puesto *No* ya que con esto evitamos problemas con posibles instrumentos a partir de dummies.

Por otro lado, repitamos lo mismo para las columnas numéricas, imputaremos 0 ya que refleja la ausencia de desempeño. Sin embargo, la columna que contiene la estadística más alta de cada medida de desempeño se imputará por separado debido a que causará problemas cuando se imponga la trnasformación para el modelo.

In [145]:
# Add string to every element in a list
# Hitter
hitter_high_stat = [stat + '_H' for stat in hitter_regular_stats]
hitter_low_stat = [stat + '_L' for stat in hitter_regular_stats]
# Fielder
pitcher_high_stat = [stat + '_H' for stat in pitcher_regular_stats]
pitcher_low_stat = [stat + '_L' for stat in pitcher_regular_stats]

In [146]:
empiric_panel_hitter[hitter_high_stat] = empiric_panel_hitter[hitter_high_stat].fillna(method = 'ffill')
empiric_panel_pitcher[pitcher_high_stat] = empiric_panel_pitcher[pitcher_high_stat].fillna(method = 'ffill')

In [147]:
empiric_panel_hitter[hitter_high_stat].head(20)

,At_bats_2_t_H,At_bats_2_t_1_H,At_bats_t_H,At_bats_t_1_H,Bateos_2_t_H,Bateos_2_t_1_H,Bateos_promedio_2_t_H,Bateos_promedio_2_t_1_H,Bateos_promedio_t_H,Bateos_promedio_t_1_H,...,Runs_batted_in_t_H,Runs_batted_in_t_1_H,Triples_2_t_H,Triples_2_t_1_H,Triples_t_H,Triples_t_1_H,WAR_2_t_H,WAR_2_t_1_H,WAR_t_H,WAR_t_1_H


In [148]:
empiric_panel_hitter.fillna(0, inplace = True)
empiric_panel_pitcher.fillna(0, inplace = True)

Verifiquemos si queda alguna columna con alguna entrada tipo *NaN*:

In [149]:
# Hitter
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [150]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

Obtengamos el máximo de cada una de las medidas de desempeño, de periodos *t_1*, por equipo que han obtenido a lo largo de todas la temporadas

Para evitar problemas con el tipo de *ID*, creemos una que sea numérica para evitar usar los nombres de los jugadores

In [151]:
# Hitter
empiric_panel_hitter['id'] =  empiric_panel_hitter.groupby(['Jugador']).ngroup()
empiric_panel_hitter.reset_index(drop = True, inplace = True)
# Pitcher
empiric_panel_pitcher['id'] =  empiric_panel_pitcher.groupby(['Jugador']).ngroup()
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

Obtengamos la transformación para obtener la $Y$ del modelo empírico a partir de los rezagos de las raices de dichos salarios

In [152]:
# Función de rezagos de raices
def sqrt_dif(X):
    S = []
    for i in range(0, len(X)-1):
        d = np.sqrt(X[i+1])-np.sqrt(X[i])
        S.append(d)
    try:
        S.append(d)
    except: 
        S.append(0)
    return S

In [153]:
Y_hitter = []
for p in empiric_panel_hitter["id"].unique():
    # Filtremos todos los sueldos (ln) de cada jugador por separado
    X = empiric_panel_hitter[empiric_panel_hitter["id"] == p]["ln_Sueldo_ajustado_t"].values
    # Aplicación de la función
    S = sqrt_dif(X)
    # Añadimos los datos de manera ordenada
    Y_hitter = np.concatenate((Y_hitter, S))
# Agregamos la columna:
empiric_panel_hitter["Y"] = Y_hitter

In [154]:
empiric_panel_hitter["Y"]

Series([], Name: Y, dtype: float64)

In [155]:
Y_pitcher = []
for p in empiric_panel_pitcher["id"].unique():
    # Filtremos todos los sueldos (ln) de cada jugador por separado
    X = empiric_panel_pitcher[empiric_panel_pitcher["id"] == p]["ln_Sueldo_ajustado_t"].values
    # Aplicación de la función
    S = sqrt_dif(X)
    # Añadimos los datos de manera ordenada
    Y_pitcher = np.concatenate((Y_pitcher, S))
# Agregamos la columna:
empiric_panel_pitcher["Y"] = Y_pitcher

In [156]:
empiric_panel_pitcher["Y"]

Series([], Name: Y, dtype: float64)

Contruyamos las dummy *I* del modelo empírico

In [157]:
# Long of the stats
end_hitter_name = int((len(hitter_high_stat) + len(hitter_low_stat))/2)

In [158]:
hitter_regular_stats

['At_bats_2_t',
 'At_bats_2_t_1',
 'At_bats_t',
 'At_bats_t_1',
 'Bateos_2_t',
 'Bateos_2_t_1',
 'Bateos_promedio_2_t',
 'Bateos_promedio_2_t_1',
 'Bateos_promedio_t',
 'Bateos_promedio_t_1',
 'Bateos_t',
 'Bateos_t_1',
 'Dobles_2_t',
 'Dobles_2_t_1',
 'Dobles_t',
 'Dobles_t_1',
 'Home_runs_2_t',
 'Home_runs_2_t_1',
 'Home_runs_t',
 'Home_runs_t_1',
 'Ingresos_equipo_2_t',
 'Ingresos_equipo_2_t_1',
 'Ingresos_equipo_t',
 'Ingresos_equipo_t_1',
 'Juegos_iniciados_2_t',
 'Juegos_iniciados_2_t_1',
 'Juegos_iniciados_t',
 'Juegos_iniciados_t_1',
 'Porcentaje_On_base_plus_slugging_2_t',
 'Porcentaje_On_base_plus_slugging_2_t_1',
 'Porcentaje_On_base_plus_slugging_t',
 'Porcentaje_On_base_plus_slugging_t_1',
 'Porcentaje_on_base_2_t',
 'Porcentaje_on_base_2_t_1',
 'Porcentaje_on_base_t',
 'Porcentaje_on_base_t_1',
 'Porcentaje_slugging_2_t',
 'Porcentaje_slugging_2_t_1',
 'Porcentaje_slugging_t',
 'Porcentaje_slugging_t_1',
 'Runs_batted_in_2_t',
 'Runs_batted_in_2_t_1',
 'Runs_batted_in_t',

In [159]:
# Hitter
for sport_stat in range(0,end_hitter_name):
    I_hitter = []
    for y,max_stat,min_stat in zip(empiric_panel_hitter[hitter_regular_stats[sport_stat]],
                                   empiric_panel_hitter[hitter_high_stat[sport_stat]],
                                   empiric_panel_hitter[hitter_low_stat[sport_stat]]):
        # Dummy condition
        if y > (max_stat + min_stat)/2:
            I_hitter.append(0)
        else: 
            I_hitter.append(1)
    
    I_name = hitter_regular_stats[sport_stat] + '_I'
    empiric_panel_hitter[I_name] = I_hitter

Veamos los resultados

In [160]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

In [161]:
# Hitter
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [162]:
start = empiric_panel_hitter.columns.get_loc('At_bats_2_t_1_I')
end = empiric_panel_hitter.columns.get_loc('WAR_t_1_I') + 1
empiric_panel_hitter.iloc[:,start:end]

,At_bats_2_t_1_I,At_bats_t_I,At_bats_t_1_I,Bateos_2_t_I,Bateos_2_t_1_I,Bateos_promedio_2_t_I,Bateos_promedio_2_t_1_I,Bateos_promedio_t_I,Bateos_promedio_t_1_I,Bateos_t_I,...,Runs_batted_in_t_I,Runs_batted_in_t_1_I,Triples_2_t_I,Triples_2_t_1_I,Triples_t_I,Triples_t_1_I,WAR_2_t_I,WAR_2_t_1_I,WAR_t_I,WAR_t_1_I


Repitamos el mismo proceso para los lanzadores

In [163]:
# Long of stats
end_pitcher_name = int((len(pitcher_high_stat) + len(pitcher_low_stat))/2)

In [164]:
# Pitcher
for sport_stat in range(0,end_pitcher_name):
    I_pitcher = []
    for y,max_stat,min_stat in zip(empiric_panel_pitcher[pitcher_regular_stats[sport_stat]],
                                   empiric_panel_pitcher[pitcher_high_stat[sport_stat]],
                                   empiric_panel_pitcher[pitcher_low_stat[sport_stat]]):
        if y > (max_stat + min_stat)/2:
            I_pitcher.append(0)
        else: 
            I_pitcher.append(1)
    
    I_name = pitcher_regular_stats[sport_stat] + '_I'
    empiric_panel_pitcher[I_name] = I_pitcher

In [165]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

In [166]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

In [167]:
start = empiric_panel_pitcher.columns.get_loc('Bateos_2_t_1_I')
end = empiric_panel_pitcher.columns.get_loc('Wins_t_1_I') + 1
empiric_panel_pitcher.iloc[:,start:end]

,Bateos_2_t_1_I,Bateos_t_I,Bateos_t_1_I,Carreras_2_t_I,Carreras_2_t_1_I,Carreras_ganadas_2_t_I,Carreras_ganadas_2_t_1_I,Carreras_ganadas_t_I,Carreras_ganadas_t_1_I,Carreras_t_I,...,WHIP_t_I,WHIP_t_1_I,Walks_2_t_I,Walks_2_t_1_I,Walks_t_I,Walks_t_1_I,Wins_2_t_I,Wins_2_t_1_I,Wins_t_I,Wins_t_1_I


Obtengamos las variables auxiliares

In [168]:
# Good practice
hitter_high_stat = sorted(hitter_high_stat)
# Dummy
I_hitter = sorted([stat + '_I' for stat in hitter_regular_stats])

Verifiquemos que tengan la misma longitud

In [169]:
len(I_hitter) == len(hitter_high_stat) == len(hitter_regular_stats)

True

In [170]:
# Pitcher
for stat in range(0,len(hitter_regular_stats)):
    # Variable auxiliar
    X_hitter = []
    
    # Variables 
    i = (-1)**(empiric_panel_hitter[I_hitter[stat]])
    x = empiric_panel_hitter[hitter_regular_stats[stat]]/np.sqrt(empiric_panel_hitter[hitter_high_stat[stat]])
    X_hitter = i*x
    
    # X name
    name = 'X_' + hitter_regular_stats[stat]
    empiric_panel_hitter[name] = X_hitter

In [171]:
# Pitcher
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [172]:
general_dynamic_path = 'ETL_Data/Panel/Cumulative/Dynamic_model/'

In [173]:
empiric_panel_hitter.to_csv(general_dynamic_path + 'panel_hitters_t_1' + '.csv',
                            index = False)

Repitamos el mismo procedimiento para los filderos

In [174]:
# Good practice
pitcher_high_stat = sorted(pitcher_high_stat)
# Dummy
I_pitcher = sorted([stat + '_I' for stat in pitcher_regular_stats])

Verifiquemos que tengan la misma longitud

In [175]:
len(I_pitcher) == len(pitcher_high_stat) == len(pitcher_regular_stats)

True

In [176]:
# Pitcher
for stat in range(0,len(pitcher_regular_stats)):
    # Variable auxiliar
    X_pitcher = []
    
    # Variables 
    i = (-1)**(empiric_panel_pitcher[I_pitcher[stat]])
    x = empiric_panel_pitcher[pitcher_regular_stats[stat]]/np.sqrt(empiric_panel_pitcher[pitcher_high_stat[stat]])
    X_pitcher = i*x
    
    # X name
    name = 'X_' + pitcher_regular_stats[stat]
    empiric_panel_pitcher[name] = X_pitcher

In [177]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

In [179]:
empiric_panel_pitcher.to_csv(general_dynamic_path + 'panel_fielders_t_1' + '.csv',
                             index = False)